<a href="https://colab.research.google.com/github/Enrico-Call/RL-AKI/blob/main/Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/master/img/logo_amds.png?raw=1" alt="Logo" width=128px/>

# VUmc Research Project - Reinforcement Learning for Sepsis Prevention
# Data Extraction

AmsterdamUMCdb version 1.0.2 March 2020  
Copyright &copy; 2003-2022 Amsterdam UMC - Amsterdam Medical Data Science

## 1. Set up the environment variables for Colab and GoogleBigQuery to access

In [1]:
import os
import psycopg2
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import io
import gc

from google.colab import auth, drive
from google.cloud.bigquery import magics
from google.cloud import bigquery

#sets dateset
DATASET_PROJECT_ID = 'amsterdamumcdb-data'
DATASET_ID = 'ams102'
LOCATION = 'eu'

#sets the project id
PROJECT_ID = "rl-aki"

#all libraries check this environment variable, so set it:
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

#sets the default query job configuration
def_config = bigquery.job.QueryJobConfig(default_dataset=DATASET_PROJECT_ID + "." + DATASET_ID, use_legacy_sql=True)
magics.context.default_query_job_config = def_config

#sets client options job configuration
client_options = {}
client_options['location'] = LOCATION
magics.context.bigquery_client_options = client_options

#BigQuery requires a separate config to prevent the 'BadRequest: 400 Cannot explicitly modify anonymous table' error message
job_config = bigquery.job.QueryJobConfig(use_legacy_sql=True)

#sets default client settings by re-using the previously defined config
client = bigquery.Client(project=PROJECT_ID, location=LOCATION, default_query_job_config=def_config)

config_gbq = {'query': 
          {'defaultDataset': {
              "datasetId": DATASET_ID, 
              "projectId": DATASET_PROJECT_ID
              },
           'Location': LOCATION}
           }

auth.authenticate_user()
print('Authenticated')

#mount drive and change it to working folder
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/MLRFH')

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Authenticated
Mounted at /content/drive


In [2]:
#get the amsterdamumcdb package from PiPy repository for use in Colab
!pip install amsterdamumcdb
import amsterdamumcdb as adb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 232 kB 14.0 MB/s 


### 1.1. Available documentation for AmsterdamUMCdb tables.
Additional information including sample Python code in Jupyter Notebooks and [SQL](https://github.com/AmsterdamUMC/AmsterdamUMCdb/tree/master/amsterdamumcdb/sql) is available from the official [AmsterdamUMCdb GitHub repository](https://github.com/AmsterdamUMC/AmsterdamUMCdb) and [wiki](https://github.com/AmsterdamUMC/AmsterdamUMCdb/wiki). 

Table | Wiki | Example code
:- | :--| -:
admissions|https://github.com/AmsterdamUMC/AmsterdamUMCdb/wiki/admissions | [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AmsterdamUMC/AmsterdamUMCdb/blob/master/tables/admissions.ipynb) 
drugitems|https://github.com/AmsterdamUMC/AmsterdamUMCdb/wiki/drugitems|[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AmsterdamUMC/AmsterdamUMCdb/blob/master/tables/drugitems.ipynb) 
freetextitems|https://github.com/AmsterdamUMC/AmsterdamUMCdb/wiki/freetextitems|[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AmsterdamUMC/AmsterdamUMCdb/blob/master/tables/freetextitems.ipynb)
listitems|https://github.com/AmsterdamUMC/AmsterdamUMCdb/wiki/lisitems|[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AmsterdamUMC/AmsterdamUMCdb/blob/master/tables/listitems.ipynb)
numericitems|https://github.com/AmsterdamUMC/AmsterdamUMCdb/wiki/numericitems|[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AmsterdamUMC/AmsterdamUMCdb/blob/master/tables/numericitems.ipynb)
procedureorderitems|https://github.com/AmsterdamUMC/AmsterdamUMCdb/wiki/procedureorderitems|[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AmsterdamUMC/AmsterdamUMCdb/blob/master/tables/procedureorderitems.ipynb)
processitems|https://github.com/AmsterdamUMC/AmsterdamUMCdb/wiki/processitems|[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AmsterdamUMC/AmsterdamUMCdb/blob/master/tables/processitems.ipynb)

**Note**: In contrast to the publicly available version of AmsterdamUMCdb, we have moved *unvalidated* (device) data (`registeredby IS NULL`) from the `numericitems` table to a separate table `unvalidatednumericitems` to reduce query costs when unvalidated device data is not needed. However, depending on the chosen solution, you may need to combine the data from both tables.

## 2. Query AmsterdamUMCdb 

Select variables of interest among patients that were diagnosed with Sepsis during their admission in the ICU at Amsterdam UMC.

Sepsis is a common reason for ICU admission. However, even though all reasons for admission system have a sepsis category, it's often not documented on that category, but instead other conditions were chosen (e.g. sepsis due to intestinal perforation will have the abdominal surgery category selected). To ameliorate this situation, the code in this notebook code uses the following definitions of sepsis:

  -  sepsis_at_admission flagged in the admission form by the attending physician
  -  the admission diagnosis, medical or surgical, is considered a severe infection, e.g., gastro-intestinal perforation, cholangitis, meningitis
  -  use of antibiotics that are not used prophylactically after surgery
  -  use of antibiotics and cultures drawn within 6 hours of admission

To do: sepsis based on Sepsis III-criteria (e.g. qSOFA)

In [3]:
#Loads the admissions table for stratification on admission unit 
admissions = pd.read_gbq(
'''
SELECT * FROM admissions;
'''
, configuration=config_gbq)

#No. patients
no_patients_total = admissions.patientid.nunique()
no_patients_ICU = admissions[admissions['location'].str.contains('IC')].patientid.nunique()
no_patients_MCU = admissions[admissions['location'].str.contains('^MC$')].patientid.nunique()

#No. admissions
no_admissions_total = admissions.admissionid.nunique()
no_admissions_ICU = admissions[admissions['location'].str.contains('IC')].admissionid.nunique()
no_admissions_MCU = admissions[admissions['location'].str.contains('^MC$')].admissionid.nunique()

combined_diagnoses = pd.read_gbq(
    '''
WITH diagnosis_groups AS (
    SELECT admissionid,
        item, 
        CASE
            WHEN itemid IN (
                18669, --NICE APACHEII diagnosen
                18671 --NICE APACHEIV diagnosen
            )
            THEN SPLIT(value,  ' - ')[SAFE_ORDINAL( 1)] 
            -- 'e.g. 'Non-operative cardiovascular - Anaphylaxis' -> Non-operative cardiovascular
            ELSE value
        END as diagnosis_group,
        valueid as diagnosis_group_id,
        ROW_NUMBER() OVER(PARTITION BY admissionid
        ORDER BY 
            CASE --prefer NICE > APACHE IV > II > D
                WHEN itemid = 18671 THEN 6 --NICE APACHEIV diagnosen
                WHEN itemid = 18669 THEN 5 --NICE APACHEII diagnosen                
                WHEN itemid BETWEEN 16998 AND 17017 THEN 4 --APACHE IV diagnosis        
                WHEN itemid BETWEEN 18589 AND 18602 THEN 3 --APACHE II diagnosis
                WHEN itemid BETWEEN 13116 AND 13145 THEN 2 --D diagnosis ICU
                WHEN itemid BETWEEN 16642 AND 16673 THEN 1 --DMC diagnosis Medium Care
            END DESC,
        measuredat DESC) AS rownum
    FROM listitems
    WHERE itemid IN (
        --MAIN GROUP - LEVEL 0
        13110, --D_Hoofdgroep
        16651, --DMC_Hoofdgroep, Medium Care
        
        18588, --Apache II Hoofdgroep
        16997, --APACHE IV Groepen
        
        18669, --NICE APACHEII diagnosen
        18671 --NICE APACHEIV diagnosen
    )
),diagnosis_subgroups AS (
    SELECT admissionid,
        item, 
        value as diagnosis_subgroup,
        valueid as diagnosis_subgroup_id,
        ROW_NUMBER() OVER(PARTITION BY admissionid
        ORDER BY measuredat DESC) AS rownum
    FROM listitems
    WHERE itemid IN (
        --SUB GROUP - LEVEL 1
        13111, --D_Subgroep_Thoraxchirurgie
        16669, --DMC_Subgroep_Thoraxchirurgie
        13112, --D_Subgroep_Algemene chirurgie
        16665, --DMC_Subgroep_Algemene chirurgie
        13113, --D_Subgroep_Neurochirurgie
        16667, --DMC_Subgroep_Neurochirurgie
        13114, --D_Subgroep_Neurologie
        16668, --DMC_Subgroep_Neurologie
        13115, --D_Subgroep_Interne geneeskunde
        16666 --DMC_Subgroep_Interne geneeskunde
    )
), diagnoses AS (
    SELECT admissionid,
        item, 
        CASE
            WHEN itemid IN (
                18669, --NICE APACHEII diagnosen
                18671 --NICE APACHEIV diagnosen
            )
            THEN SPLIT(value,  ' - ')[SAFE_ORDINAL( 2)] 
            -- 'e.g. 'Non-operative cardiovascular - Anaphylaxis' -> Anaphylaxis
            ELSE value
        END as diagnosis,
        CASE
            WHEN itemid IN (
                --SURGICAL
                13116, --D_Thoraxchirurgie_CABG en Klepchirurgie
                16671, --DMC_Thoraxchirurgie_CABG en Klepchirurgie
                13117, --D_Thoraxchirurgie_Cardio anders
                16672, --DMC_Thoraxchirurgie_Cardio anders
                13118, --D_Thoraxchirurgie_Aorta chirurgie
                16670, --DMC_Thoraxchirurgie_Aorta chirurgie
                13119, --D_Thoraxchirurgie_Pulmonale chirurgie
                16673, --DMC_Thoraxchirurgie_Pulmonale chirurgie

                --Not surgical: 13141, --D_Algemene chirurgie_Algemeen   
                --Not surgical: 16642, --DMC_Algemene chirurgie_Algemeen
                13121, --D_Algemene chirurgie_Buikchirurgie
                16643, --DMC_Algemene chirurgie_Buikchirurgie
                13123, --D_Algemene chirurgie_Endocrinologische chirurgie
                16644, --DMC_Algemene chirurgie_Endocrinologische chirurgie
                13145, --D_Algemene chirurgie_KNO/Overige
                16645, --DMC_Algemene chirurgie_KNO/Overige
                13125, --D_Algemene chirurgie_Orthopedische chirurgie
                16646, --DMC_Algemene chirurgie_Orthopedische chirurgie
                13122, --D_Algemene chirurgie_Transplantatie chirurgie
                16647, --DMC_Algemene chirurgie_Transplantatie chirurgie
                13124, --D_Algemene chirurgie_Trauma
                16648, --DMC_Algemene chirurgie_Trauma
                13126, --D_Algemene chirurgie_Urogenitaal
                16649, --DMC_Algemene chirurgie_Urogenitaal
                13120, --D_Algemene chirurgie_Vaatchirurgie
                16650, --DMC_Algemene chirurgie_Vaatchirurgie

                13128, --D_Neurochirurgie _Vasculair chirurgisch
                16661, --DMC_Neurochirurgie _Vasculair chirurgisch
                13129, --D_Neurochirurgie _Tumor chirurgie
                16660, --DMC_Neurochirurgie _Tumor chirurgie
                13130, --D_Neurochirurgie_Overige
                16662, --DMC_Neurochirurgie_Overige

                18596, --Apache II Operatief  Gastr-intenstinaal
                18597, --Apache II Operatief Cardiovasculair
                18598, --Apache II Operatief Hematologisch
                18599, --Apache II Operatief Metabolisme
                18600, --Apache II Operatief Neurologisch
                18601, --Apache II Operatief Renaal
                18602, --Apache II Operatief Respiratoir

                17008, --APACHEIV Post-operative cardiovascular
                17009, --APACHEIV Post-operative gastro-intestinal
                17010, --APACHEIV Post-operative genitourinary
                17011, --APACHEIV Post-operative hematology
                17012, --APACHEIV Post-operative metabolic
                17013, --APACHEIV Post-operative musculoskeletal /skin
                17014, --APACHEIV Post-operative neurologic
                17015, --APACHEIV Post-operative respiratory
                17016, --APACHEIV Post-operative transplant
                17017 --APACHEIV Post-operative trauma

            ) THEN 1
            WHEN itemid = 18669 AND valueid BETWEEN 1 AND 26 THEN 1 --NICE APACHEII diagnosen
            WHEN itemid = 18671 AND valueid BETWEEN 222 AND 452 THEN 1 --NICE APACHEIV diagnosen
            ELSE 0
        END AS surgical,
        valueid as diagnosis_id,
        CASE 
                WHEN itemid = 18671 THEN 'NICE APACHE IV'
                WHEN itemid = 18669 THEN 'NICE APACHE II'
                WHEN itemid BETWEEN 16998 AND 17017 THEN 'APACHE IV'
                WHEN itemid BETWEEN 18589 AND 18602 THEN 'APACHE II'
                WHEN itemid BETWEEN 13116 AND 13145 THEN 'Legacy ICU'
                WHEN itemid BETWEEN 16642 AND 16673 THEN 'Legacy MCU'
        END AS diagnosis_type,
        ROW_NUMBER() OVER(PARTITION BY admissionid
        ORDER BY 
            CASE --prefer NICE > APACHE IV > II > D
                WHEN itemid = 18671 THEN 6 --NICE APACHEIV diagnosen
                WHEN itemid = 18669 THEN 5 --NICE APACHEII diagnosen                
                WHEN itemid BETWEEN 16998 AND 17017 THEN 4 --APACHE IV diagnosis        
                WHEN itemid BETWEEN 18589 AND 18602 THEN 3 --APACHE II diagnosis
                WHEN itemid BETWEEN 13116 AND 13145 THEN 2 --D diagnosis ICU
                WHEN itemid BETWEEN 16642 AND 16673 THEN 1 --DMC diagnosis Medium Care
            END DESC,
            measuredat DESC) AS rownum
    FROM listitems
    WHERE itemid IN (
        -- Diagnosis - LEVEL 2
        --SURGICAL
        13116, --D_Thoraxchirurgie_CABG en Klepchirurgie
        16671, --DMC_Thoraxchirurgie_CABG en Klepchirurgie
        13117, --D_Thoraxchirurgie_Cardio anders
        16672, --DMC_Thoraxchirurgie_Cardio anders
        13118, --D_Thoraxchirurgie_Aorta chirurgie
        16670, --DMC_Thoraxchirurgie_Aorta chirurgie
        13119, --D_Thoraxchirurgie_Pulmonale chirurgie
        16673, --DMC_Thoraxchirurgie_Pulmonale chirurgie
        
        13141, --D_Algemene chirurgie_Algemeen   
        16642, --DMC_Algemene chirurgie_Algemeen
        13121, --D_Algemene chirurgie_Buikchirurgie
        16643, --DMC_Algemene chirurgie_Buikchirurgie
        13123, --D_Algemene chirurgie_Endocrinologische chirurgie
        16644, --DMC_Algemene chirurgie_Endocrinologische chirurgie
        13145, --D_Algemene chirurgie_KNO/Overige
        16645, --DMC_Algemene chirurgie_KNO/Overige
        13125, --D_Algemene chirurgie_Orthopedische chirurgie
        16646, --DMC_Algemene chirurgie_Orthopedische chirurgie
        13122, --D_Algemene chirurgie_Transplantatie chirurgie
        16647, --DMC_Algemene chirurgie_Transplantatie chirurgie
        13124, --D_Algemene chirurgie_Trauma
        16648, --DMC_Algemene chirurgie_Trauma
        13126, --D_Algemene chirurgie_Urogenitaal
        16649, --DMC_Algemene chirurgie_Urogenitaal
        13120, --D_Algemene chirurgie_Vaatchirurgie
        16650, --DMC_Algemene chirurgie_Vaatchirurgie

        13128, --D_Neurochirurgie _Vasculair chirurgisch
        16661, --DMC_Neurochirurgie _Vasculair chirurgisch
        13129, --D_Neurochirurgie _Tumor chirurgie
        16660, --DMC_Neurochirurgie _Tumor chirurgie
        13130, --D_Neurochirurgie_Overige
        16662, --DMC_Neurochirurgie_Overige
        
        18596, --Apache II Operatief  Gastr-intenstinaal
        18597, --Apache II Operatief Cardiovasculair
        18598, --Apache II Operatief Hematologisch
        18599, --Apache II Operatief Metabolisme
        18600, --Apache II Operatief Neurologisch
        18601, --Apache II Operatief Renaal
        18602, --Apache II Operatief Respiratoir
        
        17008, --APACHEIV Post-operative cardiovascular
        17009, --APACHEIV Post-operative gastro-intestinal
        17010, --APACHEIV Post-operative genitourinary
        17011, --APACHEIV Post-operative hematology
        17012, --APACHEIV Post-operative metabolic
        17013, --APACHEIV Post-operative musculoskeletal /skin
        17014, --APACHEIV Post-operative neurologic
        17015, --APACHEIV Post-operative respiratory
        17016, --APACHEIV Post-operative transplant
        17017, --APACHEIV Post-operative trauma

        --MEDICAL
        13133, --D_Interne Geneeskunde_Cardiovasculair
        16653, --DMC_Interne Geneeskunde_Cardiovasculair
        13134, --D_Interne Geneeskunde_Pulmonaal
        16658, --DMC_Interne Geneeskunde_Pulmonaal
        13135, --D_Interne Geneeskunde_Abdominaal
        16652, --DMC_Interne Geneeskunde_Abdominaal
        13136, --D_Interne Geneeskunde_Infectieziekten
        16655, --DMC_Interne Geneeskunde_Infectieziekten
        13137, --D_Interne Geneeskunde_Metabool
        16656, --DMC_Interne Geneeskunde_Metabool
        13138, --D_Interne Geneeskunde_Renaal
        16659, --DMC_Interne Geneeskunde_Renaal
        13139, --D_Interne Geneeskunde_Hematologisch
        16654, --DMC_Interne Geneeskunde_Hematologisch
        13140, --D_Interne Geneeskunde_Overige
        16657, --DMC_Interne Geneeskunde_Overige

        13131, --D_Neurologie_Vasculair neurologisch
        16664, --DMC_Neurologie_Vasculair neurologisch
        13132, --D_Neurologie_Overige
        16663, --DMC_Neurologie_Overige 
        13127, --D_KNO/Overige
        
        18589, --Apache II Non-Operatief Cardiovasculair
        18590, --Apache II Non-Operatief Gastro-intestinaal
        18591, --Apache II Non-Operatief Hematologisch
        18592, --Apache II Non-Operatief Metabolisme
        18593, --Apache II Non-Operatief Neurologisch
        18594, --Apache II Non-Operatief Renaal
        18595, --Apache II Non-Operatief Respiratoir
        
        16998, --APACHE IV Non-operative cardiovascular
        16999, --APACHE IV Non-operative Gastro-intestinal
        17000, --APACHE IV Non-operative genitourinary
        17001, --APACHEIV  Non-operative haematological
        17002, --APACHEIV  Non-operative metabolic
        17003, --APACHEIV Non-operative musculo-skeletal
        17004, --APACHEIV Non-operative neurologic
        17005, --APACHEIV Non-operative respiratory
        17006, --APACHEIV Non-operative transplant
        17007, --APACHEIV Non-operative trauma
        
        --NICE: surgical/medical combined in same parameter
        18669, --NICE APACHEII diagnosen
        18671 --NICE APACHEIV diagnosen
    )
), sepsis AS (
    SELECT
        admissionid,
        CASE valueid
            WHEN 1 THEN 1 --'Ja'
            WHEN 2 THEN 0 --'Nee'
        END as sepsis_at_admission,
        ROW_NUMBER() OVER(
            PARTITION BY 
                admissionid
            ORDER BY 
                measuredat DESC) AS rownum
    FROM listitems
    WHERE 
        itemid = 15808
), sepsis_antibiotics AS ( --non prophylactic antibiotics
    SELECT
        admissionid,
        CASE 
            WHEN COUNT(*) > 0 THEN 1
            ELSE 0
        END AS sepsis_antibiotics_bool,
        STRING_AGG(DISTINCT item, '; ') AS sepsis_antibiotics_given
    FROM drugitems
    WHERE 
        itemid IN (
            6834, --Amikacine (Amukin)
            6847, --Amoxicilline (Clamoxyl/Flemoxin)
            6871, --Benzylpenicilline (Penicilline)
            6917, --Ceftazidim (Fortum)
            --6919, --Cefotaxim (Claforan) -> prophylaxis
            6948, --Ciprofloxacine (Ciproxin)
            6953, --Rifampicine (Rifadin)
            6958, --Clindamycine (Dalacin)
            7044, --Tobramycine (Obracin)
            --7064, --Vancomycine -> prophylaxis for valve surgery
            7123, --Imipenem (Tienam)
            7185, --Doxycycline (Vibramycine)
            --7187, --Metronidazol (Flagyl) -> often used for GI surgical prophylaxis
            --7208, --Erythromycine (Erythrocine) -> often used for gastroparesis
            7227, --Flucloxacilline (Stafoxil/Floxapen)
            7231, --Fluconazol (Diflucan)
            7232, --Ganciclovir (Cymevene)
            7233, --Flucytosine (Ancotil)
            7235, --Gentamicine (Garamycin)
            7243, --Foscarnet trinatrium (Foscavir)
            7450, --Amfotericine B (Fungizone)
            --7504, --X nader te bepalen --non-stock medication
            8127, --Meropenem (Meronem)
            8229, --Myambutol (ethambutol)
            8374, --Kinine dihydrocloride
            --8375, --Immunoglobuline (Nanogam) -> not anbiotic
            --8394, --Co-Trimoxazol (Bactrimel) -> often prophylactic (unless high dose)
            8547, --Voriconazol(VFEND)
            --9029, --Amoxicilline/Clavulaanzuur (Augmentin) -> often used for ENT surgical prophylaxis
            9030, --Aztreonam (Azactam)
            9047, --Chlooramfenicol
            --9075, --Fusidinezuur (Fucidin) -> prophylaxis
            9128, --Piperacilline (Pipcil)
            9133, --Ceftriaxon (Rocephin)
            --9151, --Cefuroxim (Zinacef) -> often used for GI/transplant surgical prophylaxis
            --9152, --Cefazoline (Kefzol) -> prophylaxis for cardiac surgery
            9458, --Caspofungine
            9542, --Itraconazol (Trisporal)
            --9602, --Tetanusimmunoglobuline -> prophylaxis/not antibiotic
            12398, --Levofloxacine (Tavanic)
            12772, --Amfotericine B lipidencomplex  (Abelcet)
            15739, --Ecalta (Anidulafungine)
            16367, --Research Anidulafungin/placebo
            16368, --Research Caspofungin/placebo
            18675, --Amfotericine B in liposomen (Ambisome )
            19137, --Linezolid (Zyvoxid)
            19764, --Tigecycline (Tygacil)
            19773, --Daptomycine (Cubicin)
            20175 --Colistine
        )
        AND start < 24*60*60*1000 --within 24 hours (to correct for antibiotics administered before ICU)
    GROUP BY admissionid
), other_antibiotics AS ( --'prophylactic' antibiotics that may be used for sepsis
    SELECT
        admissionid,
        CASE 
            WHEN COUNT(*) > 0 THEN 1
            ELSE 0
        END AS other_antibiotics_bool,
        STRING_AGG(DISTINCT item, '; ') AS other_antibiotics_given
    FROM drugitems
    WHERE 
        itemid IN (
            7064, --Vancomycine -> prophylaxis for valve surgery
            7187, --Metronidazol (Flagyl) -> often used for GI surgical prophylaxis
            8394, --Co-Trimoxazol (Bactrimel) -> often prophylactic (unless high dose)
            9029, --Amoxicilline/Clavulaanzuur (Augmentin) -> often used for ENT surgical prophylaxis
            9151, --Cefuroxim (Zinacef) -> often used for GI surgical prophylaxis
            9152 --Cefazoline (Kefzol) -> prophylaxis
        )
        AND start < 24*60*60*1000 --within 24 hours (to correct for antibiotics administered before ICU)
    GROUP BY admissionid    
), cultures AS (
    SELECT
        admissionid,
        CASE 
            WHEN COUNT(*) > 0 THEN 1
            ELSE 0
        END AS sepsis_cultures_bool,
        STRING_AGG(DISTINCT item, '; ') AS sepsis_cultures_drawn
    FROM procedureorderitems
    WHERE 
        itemid IN (
        --8097, --Sputumkweek afnemen -> often used routinely
        --8418, --Urinekweek afnemen
        --8588, --MRSA kweken afnemen 
        9189, --Bloedkweken afnemen
        9190, --Cathetertipkweek afnemen
        --9191, --Drainvochtkweek afnemen
        --9192, --Faeceskweek afnemen -> Clostridium
        --9193, --X-Kweek nader te bepalen
        --9194, --Liquorkweek afnemen
        --9195, --Neuskweek afnemen
        --9197, --Perineumkweek afnemen -> often used routinely
        -9198, --Rectumkweek afnemen -> often used routinely
        9200, --Wondkweek afnemen
        9202, --Ascitesvochtkweek afnemen
        --9203, --Keelkweek afnemen -> often used routinely
        --9204, --SDD-kweken afnemen -> often used routinely
        9205 --Legionella sneltest (urine)
        --1302, --SDD Inventarisatiekweken afnemen -> often used routinely
        --19663, --Research Neuskweek COUrSe
        --19664, --Research Sputumkweek COUrSe
        )
        AND registeredat < 6*60*60*1000 --within 6 hours
    GROUP BY admissionid
)
SELECT 
    admissions.* 
    , diagnosis_type
    , diagnosis, diagnosis_id
    , diagnosis_subgroup
    , diagnosis_subgroup_id
    , diagnosis_group
    , diagnosis_group_id
    , surgical
    , sepsis_at_admission
    , sepsis_antibiotics_bool
    , sepsis_antibiotics_given
    , other_antibiotics_bool
    , other_antibiotics_given
    , sepsis_cultures_bool
    , sepsis_cultures_drawn
FROM admissions
LEFT JOIN diagnoses on admissions.admissionid = diagnoses.admissionid
LEFT JOIN diagnosis_subgroups on admissions.admissionid = diagnosis_subgroups.admissionid
LEFT JOIN diagnosis_groups on admissions.admissionid = diagnosis_groups.admissionid
LEFT JOIN sepsis on admissions.admissionid = sepsis.admissionid
LEFT JOIN sepsis_antibiotics on admissions.admissionid = sepsis_antibiotics.admissionid
LEFT JOIN other_antibiotics on admissions.admissionid = other_antibiotics.admissionid
LEFT JOIN cultures on admissions.admissionid = cultures.admissionid
WHERE --only last updated record
    (diagnoses.rownum = 1 OR diagnoses.rownum IS NULL) AND 
    (diagnosis_subgroups.rownum = 1 OR diagnosis_subgroups.rownum IS NULL) AND
    (diagnosis_groups.rownum = 1 OR diagnosis_groups.rownum IS NULL) AND
    (sepsis.rownum = 1 OR sepsis.rownum IS NULL) 
;
    '''
    , configuration=config_gbq)

#surgical
surgical_total = len(combined_diagnoses[combined_diagnoses['surgical'] == 1])
surgical_pct_total = 100*surgical_total/no_admissions_total

surgical_ICU = len(combined_diagnoses[(combined_diagnoses['surgical'] == 1)  & combined_diagnoses['location'].str.contains('IC')])
surgical_pct_ICU = 100*surgical_ICU/no_admissions_ICU

surgical_MCU = len(combined_diagnoses[(combined_diagnoses['surgical'] == 1)  & combined_diagnoses['location'].str.contains('^MC$')])
surgical_pct_MCU = 100*surgical_MCU/no_admissions_MCU

#medical
medical_total = len(combined_diagnoses[combined_diagnoses['surgical'] == 0])
medical_pct_total = 100*medical_total/no_admissions_total

medical_ICU = len(combined_diagnoses[(combined_diagnoses['surgical'] == 0)  & combined_diagnoses['location'].str.contains('IC')])
medical_pct_ICU = 100*medical_ICU/no_admissions_ICU

medical_MCU = len(combined_diagnoses[(combined_diagnoses['surgical'] == 0)  & combined_diagnoses['location'].str.contains('^MC$')])
medical_pct_MCU = 100*medical_MCU/no_admissions_MCU

#surgical/medical unknown
medical_surgical_na_total = len(combined_diagnoses[pd.isnull(combined_diagnoses['surgical'])])
medical_surgical_na_pct_total = 100*medical_surgical_na_total/no_admissions_total

medical_surgical_na_ICU = len(combined_diagnoses[pd.isnull(combined_diagnoses['surgical']) & combined_diagnoses['location'].str.contains('IC')])
medical_surgical_na_pct_ICU = 100*medical_surgical_na_ICU/no_admissions_ICU

medical_surgical_na_MCU = len(combined_diagnoses[pd.isnull(combined_diagnoses['surgical']) & combined_diagnoses['location'].str.contains('^MC$')])
medical_surgical_na_pct_MCU = 100*medical_surgical_na_MCU/no_admissions_MCU

#urgency
urgency_total = len(combined_diagnoses[combined_diagnoses['urgency'] == '1']) #apparently the postgresql bit become an object
urgency_pct_total = 100*urgency_total/no_admissions_total

urgency_ICU = len(combined_diagnoses[(combined_diagnoses['urgency'] == '1')  & combined_diagnoses['location'].str.contains('IC')])
urgency_pct_ICU = 100*urgency_ICU/no_admissions_ICU

urgency_MCU = len(combined_diagnoses[(combined_diagnoses['urgency'] == '1')  & combined_diagnoses['location'].str.contains('^MC$')])
urgency_pct_MCU = 100*urgency_MCU/no_admissions_MCU

#Surgical

re_cardiosurg = r'(CABG|AVR|hartchirurgie|heart surgery|Chron. cardiovasculaire ziekte|hartkleppen' + \
r'|cardiovascula|MVP|MVR|mitral|tricuspid|pericard|aortic.*valve|lobectom|segment|thorax|Bentall|aorta-ascendens|aorta-boog' + \
r'|aorta-wortel|aorta-descendens|lung|pneumectomie|bullectom|respiratoir neoplasm|thoracoscop|thoracotom(y|ie)|respirato' + \
r'|vrije wand ruptuur|VSR|ASD|pleurectom|intracardiac|aneurysmectom|congenital defect repair)(?! for esophag)'
re_neurosurg = r'neuro|tentorieel|cranial|subdur|epidur|subarachn|intracerbr|hoofdtrauma|SAB|S.A.H.|CNS|Hoofd|seizures' + \
r'|craniotom|cranioplast|spinal|dwarslaesie|ventriculstom|transphenoidal|brain|A.V.M.|Arteriovenous malformation' + \
r'|brughoek|shunts and revisions|stereotactic|Burr hole|cerebrospinal'
re_vascsurg = r'vaatchirurgie|vasc.*surg|EVAR|aorta vervanging|perifeer vasc|embolectom|aneurysm|carotid|endovasc' + \
r'|dissectie|endarterectom|thrombectomy|dilatation|PTCA|all other bypass|femoral-popliteal|aorto-femoral|femoral-femoral' + \
r'|transplantectom'
re_gisurg = r'oesophagus|esophageal|maag|stomach|darm|intestin|gastro-intestin|pancreatitis|laparotom|gastro-intestinale perforatie' + \
r'|galblaas|Bleeding-.*GI|other GI|colon|rectal|GI.*surgery|GI obstruction|Whipple|diverticular|appendectomy' + \
r'|peritonitis|cholecystectomy|exenteration'
re_uro = r'(?<!ne)(urolog|cystectomy|genitourinary surgery|prostatectom|ileal\-conduit|orchiectomy|bladder repair' + \
r'|nefrectom|nephrectom|renaal neopsplama)'
re_obgyn = r'hysterectom|Cesarean|ectopic pregnancy'
re_gensurg = r'mond/keel|cancer oral|cancer laryngeal|spondylodes|Fusion-spinal|devices for spine|orthopedic' + \
r'|renaal|metabol|endocrin|thyroid|hip replacement|knee replacement|adrenalectom|tracheostomy|bewaking|amputation|apnea-sleep' + \
r'|lymph node dissection|mastectomy|cosmetic|fracture-pathological|bewaking'
re_trauma_surg = r'(?<!non-)(?<!see )(trauma|hypotherm|smoke inhalation)(?!, see trauma)(?! see)(?!: see)'
re_tx_surg = r'niertransplantatie|kidney transplant|renaal transplantatie|pancreastransplantatie'

re_respfailure_surg = r'resp.*insuff|na respiratoir arrest|arrest, respiratory|atelectas|pneumoni|na ok'

re_sepsis_surg = r'sepsis|pneumoni|GI perforation|perforation/rupture|infection|abscess|GI Vascular ischemia|diverticular|appendectomy|peritonitis'
re_cardiacarrest_surg = r'Cardiac arrest.*respiratory arrest|na reanimatie'
re_hepa = r'lever'
re_surg_other = r'diagnose anders|respiratoir|cardiovasculair|niet operatief'

re_surgical_medical = r'Bloeding tractus digestivus|Haemorragische shock|Gastro-intestinale bloeding|Bleeding, upper GI' + \
r'|hematologisch|Hematologische maligniteit|Haematologisch|Renaal'

re_surgical = '(' + \
    re_cardiosurg + '|' + \
    re_neurosurg + '|' + \
    re_vascsurg + '|' + \
    re_gisurg + '|' + \
    re_uro + '|' + \
    re_obgyn + '|' + \
    re_gensurg + '|' + \
    re_trauma_surg + '|' + \
    re_tx_surg + '|' + \
    re_hepa + '|' + \
    re_surg_other + '|' + \
    re_surgical_medical + '|' + \
    re_respfailure_surg + '|' + \
    re_sepsis_surg + '|' + \
    re_cardiacarrest_surg + '|' + \
    ')'

#check to see if we caught all (should return empty data frame):
combined_diagnoses[(combined_diagnoses['surgical'] == 1
) & ~(combined_diagnoses['diagnosis'].str.contains(
        re_surgical, na=False, flags=re.IGNORECASE))].groupby(['diagnosis_group','diagnosis']).size().to_frame('number of admissions').sort_values('number of admissions', ascending=False).reset_index()

#Medical

re_respfailure_med = r'(?<! without )(resp.*insuff|pneumoni|respirato|luchtweg obstructie|obstruction-airway' + \
r'|chronisch obstructieve longziekte|emphysema|asthma|aspiratie|aspiration|longembolie|pulmonary|pulmonaire' + \
r'|long|lung|atelectas|ALI|ARDS|pleural|cancer, lung|pneumothorax|verdrinking|near drowning|weaning|hemothorax|PCP)'
re_cardiacarrest_med = r'Cardiac arrest.*respiratory arrest|na reanimatie'
re_sepsis_med = r'sepsis|septic|infect|pneumoni|cholangitis|pancr|endocarditis|meningitis|GI perforation|abces|abscess|darm ischaemie|GI vascular|fasciitis' + \
r'|inflammatory|peritonitis'
re_cardio = r'cardiogene shock|shock, cardiogenic|angina|ritme|rhythm|cardiovascular|cardiovasculair' + \
r'|myocardial|endocarditis|coronair|coronary|cardiomyopath|tamponade|pericardial|papillary muscle|^MI|hartkleppen' + \
r'|CHF|hart falen|decompensatio cordis'
re_neuro = '(?<!see )(insult|seizure|CVA|observatie neurologische status|intracerebraal haematoom|intracranial|intracerebr|subdur|subarachno' + \
r'|epidur|coma|neurologisch|neurologic|CZS|S\.A\.B\.|neurologie|hoofdtrauma|head|neuro|muscula|spinal|meningitis|encephalitis' + \
r'|myasthenia|vaatspasme protocol|Guillian-Barre|encephalopath|musculoskeletal)(?!-see Neurological System)'
re_bleeding = r'bloeding tractus digestivus|gastro-intestinale bloeding|gastro-intestinaal|bleeding, upper GI|bleeding, lower GI' + \
r'|bleeding, GI|ulcer'
re_gi_med = r'ileus|GI medical|GI obstruction'
re_tox = r'intox|overdosis|overdose|toxicity|withdrawal|drug'
re_trauma_med = r'(?<!non-)(?<!see )(trauma|hypotherm|smoke inhalation)(?!, see trauma)(?! see)(?!: see)'
re_hemonc = r'malign|hematolog|cancer|bloeding|TTP|HUS|leukemi|pancytopen|coagulopath|anemia|neutropen|lymph|sickle'
re_endo_med = r'metabolisme|keto-acidose|diabetic|metabolic|endocrine|hypertens|acid-base|hypoglyc|thyroid'
re_shock_med = r'hypovolemische shock|shock|haemorr|hemorrha|anaphylaxis'
re_nefro_med = r'renaal|renal|tubulus|genitourinary|urolog|nefr'
re_hepa_med = r'lever|hepatic'
re_obgyn_med = r'obstetrie|postpartum|eclampsia'
re_vasc_med = r'aneurysm|vascular|cardiovascular medical|Thrombus,arterial|vascular medical|dissect|EVAR|embolectom'
re_mon_med = r'apnea, sleep|monitoring|bewaking|observatie'
re_tx_med = r'transplant'

re_medical = '(' + \
re_respfailure_med + '|' + \
re_cardiacarrest_med + '|' + \
re_sepsis_med + '|' + \
re_cardio + '|' + \
re_neuro + '|' + \
re_bleeding + '|' + \
re_gi_med + '|' + \
re_tox + '|' + \
re_trauma_med + '|' + \
re_hemonc + '|' + \
re_endo_med + '|' + \
re_shock_med + '|' + \
re_nefro_med + '|' + \
re_hepa_med + '|' + \
re_obgyn_med + '|' + \
re_vasc_med + '|' + \
re_mon_med + '|' + \
re_tx_med + '|' + \
')'

#check to see if we caught all (should return empty data frame):
combined_diagnoses[(combined_diagnoses['surgical'] == 0
) & ~(combined_diagnoses['diagnosis'].str.contains(
        re_medical, na=False, flags=re.IGNORECASE))].groupby(['diagnosis_group','diagnosis']).size().to_frame('number of admissions').sort_values('number of admissions', ascending=False).reset_index()

#Extract patients that had a diagnosis of Sepsis

sepsis = combined_diagnoses[
    (
        (
            #use reasons for admission
            #surgical admissions with sepsis
            (combined_diagnoses['surgical'] == 1) 
            & (combined_diagnoses['diagnosis'].str.contains(re_sepsis_surg, na=False, flags=re.IGNORECASE))
        ) | (
            #medical admissions with sepsis
            (combined_diagnoses['surgical'] == 0) 
            & (combined_diagnoses['diagnosis'].str.contains(re_sepsis_med, na=False, flags=re.IGNORECASE))
        ) | (
            #uses documentation at admission form (Early Goal Directed Therapy)
            (combined_diagnoses['sepsis_at_admission'] == 1)
        ) | (
            #uses administered (therapeutic) antibiotics for determining sepsis
            (combined_diagnoses['sepsis_antibiotics_bool'] == 1)
        ) | (
            #uses combination of administered antibiotics (that sometimes are used as prophylaxis) AND 
            #drawn cultures for determining sepsis
            (combined_diagnoses['other_antibiotics_bool'] == 1) 
            & (combined_diagnoses['sepsis_cultures_bool'] == 1)
        )
    ) & ~(combined_diagnoses['sepsis_at_admission'] == 0) 
    #exclude all diagnoses where explicitly 'no sepsis' was documented
]
sepsis_counts = sepsis.fillna('N/A').groupby(['diagnosis_group','diagnosis']).size().to_frame('n_admissions').sort_values(
    'n_admissions', ascending=False).reset_index()
sepsis_counts['n_admissions'].sum()

# sepsis.reset_index(inplace=True)
admission_ids = sepsis['admissionid'].to_string(index=False)
admission_ids = admission_ids.replace('\n', ',')
patient_ids = sepsis['patientid'].to_string(index=False)
patient_ids = patient_ids.replace('\n', ',')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:508: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:536: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [4]:
# %%bigquery state_space_query
# WITH adm AS(
#   SELECT 
#     a.admissionid,
#     a.weightgroup,
#     a.heightgroup,
#     a.gender,
#     a.agegroup,
#     a.admittedat, 
#     a.patientid,
#     a.dateofdeath
#   FROM admissions a
#   WHERE a.admissionid IN (6720,17191,17961, 4310,13925,12231, 1888,11884,  480,13611,  332,  125, 9284, 8865, 9651,12007,12008,10357, 1121, 3784, 7859,10358,15320,17652,19043,20137,20722, 2164,22537, 1927, 3958, 9406,11308,16789,20980, 9467,  751, 1317, 9092,23540,  460,  512, 9927,11542,15224, 3658, 7620, 9020,13441,16369,19066,21996, 8401,17696,14984,19130,22849, 1375, 1965, 1980, 2754, 3326, 3458, 4109, 4812, 4940, 5769, 6471, 7003, 8383, 8573,10350,14155,14571,17341,18339,19082,19134,19615,19689,19847,20219,21083,21247,21857,11319,22262, 2954, 3568, 6195, 8830,15681,15808,17077,20596, 1402, 4311, 5649, 6982, 2994, 4639,18759,22387,10771,19898, 4523,  223, 2049, 4429, 6590, 7772, 9986,10764,11483,12977,15204,15718,17264,17655,22427,22658, 2300,12733,23276,  252, 1914, 2215, 3073, 3105, 3333, 5225, 5832, 6959, 7351, 8295, 9041, 9810,10016,11069,11462,12141,13060,14512,17190,19663,   20,  208,  467,  733,  884,  892, 1084, 1108, 1677, 1692, 1900, 2193, 2307, 2381, 2397, 2494, 2692, 2933, 2953, 2982, 2999, 3365, 3558, 3630, 3802, 3831, 3874, 3891, 3908, 4002, 4080, 4193, 4347, 5164, 5525, 5908, 5979, 6034, 6131, 6158, 6242, 6251, 6673, 6763, 6862, 7254, 7441, 7841, 8043, 8652, 8753, 8814, 8936, 9252, 9254, 9345, 9471, 9546, 9849, 9871,10154,10553,10794,11079,11259,11289,11345,11905,11912,12113,12413,12565,12592,12660,12672,12781,12850,13182,13309,13346,13385,13421,13571,13681,13744,13846,13957,14175,14246,14314,14468,14549,14573,15489,15490,15610,15695,15738,15809,16048,16144,16161,16275,16341,16449,16707,16937,17770,18060,18070,18165,18188,18316,18413,18462,18509,18513,18652,18660,18696,18728,19069,19117,19241,19732,19886,19897,20176,20263,20493,20697,20854,20861,20942,21270,21450,21616,21858,21891,21973,21989,22038,22476,22571,22631,22655,22709,22719,22852,22874,23001,23212, 1680, 2755, 2879, 5928, 6310,10060,13348,13440,16422,17848, 2462, 2265, 2435, 2583, 3546, 7246,13315,15673,16562,  939, 5564, 6481, 7409,11858,14493,15845,15938,17506,19477,22780,22890,23050, 2411, 3074, 3152, 3627, 4168, 4731, 5871, 6033, 6411, 6589, 8298, 9605,11539,11828,12305,13802,15037,16513,16561,17562,18199,19478, 3368, 7284, 7285, 8705,13073,22202,22639,20591,  644, 2984, 3220, 3376, 3426, 3549, 3949, 4633, 5906, 6214,16013,16384,16388,16843,17111,19346,23373,  740, 7650, 8469, 8648, 9506,12126,16506,17360,19482,13261,  564, 1614, 5219, 6765, 7277, 9564, 9703,12956,15381,20545,20630,13532, 9895,10914, 1482, 2838, 3980, 5470, 6848, 7251,11920,21068,  446,  858,  953, 1296, 2010, 2845, 3190, 3319, 4484, 5498, 5656, 5697, 5897, 6551, 7059, 7981, 8427,10178,10600,12118,14474,14775,16211,18657,18673,21164,21781,22682,23150, 5840, 7667,15871,22699,  421, 5851,10896,11682,13814,14433,15304,20399,20558,21325,22142, 1309, 2589,16360,19000,19279, 2935,  159,  186,  243,  559,  801,  879,  888, 1679, 1756, 2264, 2365, 2479, 2648, 2750, 2791, 3010, 3327, 3450, 3735, 4038, 4355, 5387, 5651, 5711, 5721, 5937, 5964, 6151, 6736, 7047, 7376, 7726, 7931, 7985, 8088, 8279, 8367, 8521, 8562, 8717, 9194, 9818, 9842,10074,10644,10744,11002,11300,11433,11524,11675,11799,11866,11989,12042,12290,12318,12634,12806,12853,12866,12990,13063,13074,13239,13885,13963,14291,14382,14539,14887,15366,15383,16087,16378,16470,16632,16818,16875,17013,17479,17664,17694,17830,17833,17840,17931,18028,18131,18294,18308,18668,18863,18940,19590,19625,20307,20364,21028,21361,21657,22056,22356,22618,22674,22893,23031,23173,23328,23335,23336,23468, 9899,13072, 7662, 6582,13730,21710, 2600, 6970, 9717,12159,13761,17261,21895,23166, 8749,15937, 9636,15981,21934, 1374, 6253, 6898,10344,12158,17785,20584, 7256, 9146, 8874,10539,11239,15455,16996,22000,22340, 6860,21161, 1827,   25, 3905, 5251, 5829, 6122, 6640, 7504,14940,17629,20868,  576,  867, 4711, 8197, 8442, 8929,10023,14931,15442,15531,17651,17853,18622,19315,19970,20985,21059,21287, 1274, 6288,19262, 1845,15672,17168, 2807,18573, 1669, 3780, 5950,10056,10665,10735,13095,16061,16776,17025,17919,18678,22453,22966, 6377,18949,23388, 3031, 5347, 9922,13246,13929,14485,15805,16625,18734,21963,   70,  344,  502,  789,  988, 1003, 1392, 1758, 1804, 2096, 2550, 2604, 3405, 3758, 4094, 4202, 4837, 5000, 5113, 5170, 5263, 5477, 5848, 6421, 6865, 7286, 7680, 7937, 8740, 8765, 9368, 9597, 9654, 9914,10065,10121,10146,10208,10304,10409,10623,10909,10947,11051,11594,11612,11686,11916,11959,11993,12524,12697,12719,12740,12851,12991,13360,13480,13502,13738,13796,13984,14044,14192,14359,14557,14620,14941,15059,15064,15512,15771,15852,15991,16041,16280,16442,16454,16676,16689,16995,17150,17167,17612,17627,17638,17691,18646,18989,19435,19752,19856,20014,20344,20823,21228,21478,21808,22245,22313,22494,22568,23165,23225, 3362, 8261,14338,16683,17619,18393, 1353, 5089, 9255,12756,17903,19144,23470, 1135, 3136, 8691,11143,14939,17321,18504,19866,22823,23201, 4241, 7602, 8259,11293,12574,20388,20618,21571, 2482,15675,17079,21770,22608,  647, 4358,11807,13589,14344,15232,23462,22884, 5120, 5187,16758,22600,22767,23545,  213,12640,21243,23243, 1691, 2231, 6234,12981,18401,18898,21796,  560, 2151, 3579, 3785, 4886, 6102, 7689, 7863, 7988,13645,14646,17115,20115,  441, 3295, 4879, 7800, 8864, 9245,13517,13944,16365,16452,17459,22070,22603, 6645,   11,  165,  229,  245, 1602, 1613, 1727, 2024, 2237, 2285, 2346, 2581, 3160, 3300, 3390, 3840, 3997, 4015, 4163, 4500, 4535, 4668, 4673, 5008, 5019, 5335, 5442, 5557, 5789, 6055, 6238, 7235, 7460, 7823, 8041, 8096, 8594, 9382,10193,10401,10414,10843,10953,11177,11421,11893,11991,12211,12437,12922,13174,13299,13471,13545,13714,13766,14184,14339,14423,14630,14656,14805,14858,15086,15347,15662,15666,15827,16534,16656,16711,17049,17391,17517,17866,17922,18289,18517,18576,18725,18820,19436,19708,19950,20081,20211,20225,20411,21685,21752,21822,21860,21880,22144,22173,22435,22529,22673,22911,23341, 2829,14960,18368,19064,19424,22004,11690,  218,  685, 1262, 8255,14064,20083, 4799, 5107, 7010,11192, 9490,18451,  146,  859, 2350, 2835, 3129, 4921, 5203, 5866, 6100, 6983, 6990, 7786, 8802, 9410, 9889,11071,11341,14134,14335,16340,16627,16911,17936,22368,22916,   94, 1332, 2465, 5011, 5728, 6638, 6833, 6916, 7406, 7531, 7897, 8076, 8961, 9576, 9704,10036,11384,11445,11566,12492,13459,13719,13978,14615,16246,16326,16495,18522,19701,19727,19821,19966,20430,20976,21188,22261, 2277, 6152,  997, 1987, 4085,15886,19533,20323,  620, 2777, 2907, 7308,13269,20928, 1635, 2082, 2562, 6926, 6966, 7151, 8348, 8363, 9275,10009,12460,16230,21195,21865,22286, 5054, 8232,12960,14307,14545,23000, 1797, 6442, 8460, 9352,11265,11444, 8374,11408,11223, 4186,21286, 1070, 1449, 2580, 3715, 5579, 7630, 7643, 9670, 9774,11299,11579,13539,15317,15422,17852,18768,19354,19430,19444,20725,21466,21733,21803,21835,22518,23214,13835,16435, 3710, 4023, 6931, 8121, 8249,10166,10899,11017,15229,20993,  643, 7104, 8793, 9119,12193,12311,16244,18489,   66, 1115, 1180, 2980, 3989, 4770, 4802, 6053, 7669, 8212,11283,12345,13260,14363,14533,15877,15932,17078,17152,17662,17897,18109,18905,19521,21868,23164,10586,18812,19818,13387,  133, 8556, 9388,11090, 1278,21856, 2301, 2849,14342,20311,21824,12415, 1755, 2214, 9239,21442,  666, 2658, 5218, 7060,12473,13890,14670,17695,19267,20457,20851, 8453,15817,17058,20643, 4594, 6094,14437,15299,15961,19471,20313, 3396, 3682, 5531,21258,17498, 9535,14074, 2028, 3483, 3042,16276, 7340, 4350, 5409,12130,15501,20909,14499,15292, 1106, 3984, 6454, 9291,10918,11227,11736,12551,17716,18119,19748,22817, 5777,18787,22611,22837,20982, 1367,  348, 2189, 3918, 5775, 6635, 8990, 9181,11007,11570,13124,14072,20890,22927,18265,22998, 3928,15017,21642,19041, 8158,13755,16490,19277, 4505,15567, 4602, 5627,10512,14216,15363, 4042, 4622, 5221,11668,17403,  270, 1219, 8118,17469,17983,21683,22779,17823,17860,16915,17729,22132, 7905,14418,22348, 5619,12965, 4177, 6415, 7564,10214,23302, 2325, 3876,15914,14487,12933,18723,21277, 6210, 6642, 9494,12579,13704,20562, 2084, 3521, 8136, 1455, 9076,14632,14798,16090,22466, 4178,19202,22757, 5622, 6941,21647, 8321,11665,17760,17726, 5827, 9217,21155, 1595, 2893, 3016, 3661, 3706, 4715, 6112, 6181, 8634, 9693,11354,12932,13198,13458,14548,14851,16590,16778,17002,17672,19565,21626,22718, 6895, 6958,14142,  265,17100,11753,10992,12874, 2044, 2888, 3509, 3843, 5623, 6919, 7506, 7522, 9900,10446,10482,12723,13011,13991,15394,15571,15647,16165,17125,18068,18756,19155,12319,12542,23062, 9509, 4944,18206, 9385,14113,  203, 1310, 2054, 2657, 3301, 5740, 6924, 7349, 7632, 9852,11250,11465,12346,15680,17831,19804,22831,22832,13518,20272, 4791,12961, 2559, 2634, 5725,21754, 5540, 7293, 8853, 9397,10382,18873,20720,21753,  204, 7616, 7423,  495, 6945, 8618,10253,10331,12688,12734,13066,18633,20102,21049,21213,21406,22353, 3882, 1661, 3981,15557,18906,21533,12303, 4246, 4632, 5300, 8395,14675,15978,18959, 2422,15488,10532,12899, 5696, 7067, 7899,17951,15245,  225, 4216,12509,12510, 5299, 1032, 1361, 6709,17499, 8660,  188,  492,  992, 1437, 1522, 3296, 4034, 4398, 5736, 6235, 6351, 6793, 7432, 7839, 8035, 9007,10510,11729,11756,12974,13243,13642,15278,15849,16160,16206,17635,18150,18155,18291,18643,19582,19738,19814,19844,20005,20933,21810,22478,22543,23420, 5757,19790,22031,17071,  273,  274, 1112, 1456, 1590, 1715, 1724, 1732, 1809, 1958, 2368, 2402, 2669, 5127, 8054, 8560, 8927, 9438, 9804,10658,11119,11359,13087,13200,13489,13879,14518,16015,16836,16837,16883,17856,18407,19052,19074,19388,20557,21742,22771,22830,23361,23482, 1149, 2840, 8196, 9929,12296,15248,18807,21038, 8423, 7682,  210,  514, 3966, 4854, 7214, 7444, 8110,13617,16093,17389,18721,21134,22995, 1546, 1947, 6683, 7529, 7996,18140,19323,22454,22860,  720, 5560, 7056,15127,20066, 4205,10635,18298, 1315, 6355,14078,  676,  970, 1946, 4405, 4576, 5152, 5334, 5584, 5692, 6320, 6887, 7198, 7205, 8007, 8320, 8494, 8921, 8926, 9370, 9479,10150,11270,12578,13692,13933,14171,14631,14695,15423,16557,16720,18008,18292,18612,20138,20755,20768,21760,21906,22221,22267,22627,22717,23363,23488, 2052, 3856,13977,23190, 2952, 5003, 5572,10021,11559,  918, 9154,11982,20667,20816,17183,22548,23406,  977, 3756, 8382, 8440, 8848, 8857, 9244, 9801, 9976,10500,10930,12263,13251,13528,14882,14963,15910,18677,18699,18913,19542,20174,22160,13170, 2540, 4210, 8519,  863, 1434, 3018, 3598, 3814, 4925, 5153, 5573, 5706, 5716, 5802, 7949, 8021, 9726, 9742,10119,11778,11837,12023,14181,14994,16152,17084,17395,18749,19051,20216,20255,20516,20677,22166,22845,23472, 3159, 6853, 8751,11142,13783, 6331,17550,15275,17390,16138,  917, 2108, 3019, 3823, 4142, 7363,10845,11797,19091, 1457, 1681, 3686, 6604,11247,11409,11486,15578,16267,20268,22566,23282,23458, 7534,10622,22552,  645, 8145,13824,16770,17463, 1429, 9193,19329,22009,15942,  442,  919, 2789, 3699, 6016, 6499,10145,10317,10552,12645,12685,16301,17052,20341,20437,21727,23483, 9148, 9634,16304,17467,20356,20704,16820,17387,23140, 3742,15605,19356,  299, 8500, 9779,12897,13880,13903, 4189,13228,15370,12284, 7641,12654, 7177,20838,11307, 3533,19288,22145, 2070, 4923,16914, 6936,11059, 1674, 1169, 4083, 4524, 5624,10445,10524,11235,18415,22668,23193,   99, 1660,15466,15881,16850,17074,21396,21959,22641,23514,  774, 3857, 4353, 4572, 5753, 7482, 7599, 7888, 8592, 9010, 9912,11832,12371,12661,14115,15548,15555,15558,16549,17243,19494,19583,19805,20517,20602,21755,23304,17170, 3974, 7305,17673,16111,21209, 1348,15597, 3893, 8729, 8974,11145, 7663, 9903,11495,15566,16668,21515,22238, 1260, 1403, 3058, 5873, 6261, 6409, 6803, 7877,10235,10951,13013,13784,16329,17594,19138,20355,20413,20476,21119,22455,22672,22698,14165,15660,16173,20574,22501, 7468, 8174,10080,10215,15921,19938,11823,11970,16701, 4859, 4992, 9886,10342,20250, 1252, 3202, 3323,11072,18133,22207,10087,12003,16217,18950,18951,20811,13481,  318,  479,  976, 1411, 1847, 2115, 3059, 3111, 5283, 6271, 8142, 8963, 8977, 9463, 9473, 9748,10217,10625,11768,11824,12839,13020,13190,14389,14392,14397,14758,15694,16730,17134,17242,17428,17489,17883,18127,18714,18954,18955,19372,19495,20231,20293,21022,21401,21827,23171,11898,13953, 4835,15518,18249, 8216,15191,15632, 2457,12258,12313,13626,17314,19334, 3520,12662, 6436, 8250, 1502, 2046, 2569, 3986, 7025, 8945,10929,12844,16070,16552,21172,12255, 5397, 9328, 9530,11013,13788,22721, 2099, 4906,19395,  877, 2821, 5128,13221, 8677,15717,  272,  310, 2779, 3112, 3563, 4627, 4691, 4855, 4991, 5803, 5952, 7431, 7558, 9841, 9910, 9968,11095,11205,11551,12310,13040,13128,14770,15253,15529,16759,18290,18421,18882,19224,20106,22169,22687,23128,23372, 1102, 1237, 2284, 2946, 4019, 4053, 5034, 5069, 6714, 6826, 7554, 7976,14086,15454,15867,16468,16624,19940,20088,20573,20964,21130,21487,21873,22239,22887,12298,16533,  134,  547,  584,  800, 1013, 1018, 1105, 1123, 1547, 1688, 1754, 2483, 2769, 3068, 3140, 3336, 3656, 3855, 3910, 4065, 4696, 5427, 5975, 6050, 6276, 6668, 6676, 7019, 7021, 7134, 7319, 7337, 7370, 7576, 7914, 8005, 8247, 8372, 8517, 8540, 9107, 9206, 9247, 9349, 9440, 9512, 9662,10042,10242,10647,11025,11201,11260,11496,11666,11917,12577,12619,13090,13367,13734,13762,14055,14130,14145,14193,14421,14621,14643,14649,14876,14918,15396,15424,15617,15645,16201,16827,16967,17153,17241,17558,17762,17975,17995,18221,18790,19008,19090,19690,20011,20199,20251,20482,20812,20953,21168,21292,21392,21548,21611,21927,22104,22222,22257,22467,22694,22919,22923,23211,23476,23535, 1172, 8631,11248,12292,14700,17222,20107,20887, 3097, 2093, 6914,17182, 1513, 7391,14059,18010,20622,20139,20623,23186, 4476,12243,14018, 5401,12770,14496,15158,19245,20425,  108, 2050, 2466, 5695, 6820, 6883, 7402, 7447, 7774, 7955, 9182,10058,11769,12341,12529,12760,13955,14552,15528,15556,15616,17297,18661,18715,21510,22848,  957, 4624, 5100, 7474, 4766, 6675,12758,14469,   44,  896, 1027, 1464, 3008, 3498, 4561, 5284,10348,13562,14257,16168,19629, 5905, 6223, 8001, 8815,10368,12972,23480, 5542, 1336, 8417,11761,16017,17723, 2606, 3720, 8690,12192,12530,13273,16623,19563,  493,11748,21580,14896, 7040,14647,16448,20132, 9136,10505,14427,16941,21140,11006,20817,  160, 2147, 3006, 3783, 4889, 6071, 6196, 7961, 8122, 8686, 9806,10006,10976,11240,11634,13653,13902,15117,15706,16009,16248,17218,17720,18444,18869,22273,23155,23188, 1311, 2445, 2820, 2989, 4446, 4781, 5913, 6121, 8575, 8930,10112,10549,11082,11757,12244,13950,15111,15439,15517,17566,18009,18031,18301,23461,16129,   50,   91,  904, 1246, 1465, 2041, 2148, 2266, 3481, 3616, 4842, 4908, 5108, 5574, 5894, 6370, 6844, 7417, 7969, 8569, 8687, 8688,10323,10461,10829,11340,11691,11723,11947,12707,12761,13220,13521,13667,13690,13867,14019,14396,14479,14490,15081,15159,15446,15513,16427,17480,18058,18302,18446,18834,19164,19857,21465,21489,22134,22184,18597, 7623,10124, 5779, 1824, 2833,17849, 8277, 2473, 9409, 6641,18447, 3428, 4970,10943,  600, 2795,22862,14457, 1655, 1656, 1775, 4743,11150, 3056,20819,  677, 1879, 4346,16132,19610,20583,13934,15912,19081,  775, 7875, 4409, 7116, 3816,13410,22278, 2136,17082,12920, 9824,12449, 2205, 7379,17034,19253, 8187, 8643,12963,13700,22097,20585, 2579, 1749, 2188, 5458, 6201, 9322,11216,17466, 6546, 3076,11882,18582,  364,17616, 2955,12448,12927,18523,18391,19526, 1739,14090,22950,18870,14141, 8764,10326,19366, 6950, 1713, 9204,12336,19383, 4313,10343,13826,16517,  144, 7103,10341,10937, 6710,10597,21460, 1289, 2125, 3079,10088,10106,12281,12353,14862,16007,16612,17371,18216,19062,19717,21894,23382,11351,18347, 5382,14292, 6120,10000, 1618, 2853, 4410, 5206, 5439,12259,15611,15945,16733,18205,21154,21414,23075, 8834,11365,  403, 2553, 4805, 5440, 6005, 7446, 8741,12221,12470,13660,16672,21259,22596,22898, 5355,11685, 3104,10865,21874,18825, 1665,15713,17997,20053,20117, 3055,19217,19450, 1530, 1852, 2366,18225,18226, 7142,  387,18050,21199,  246, 5780, 6046, 7833, 8429, 8971,10051,10677,11010,11163,12246,20795,22805,22901,11349,13622, 1053, 7191,10381, 4727,11942,  901,  902, 5373, 6179,10395,14810, 5177, 5638, 7317,11490,12823,13270,14346,17407, 5313,  424, 2061, 2146, 2698, 3959, 5504, 7473, 9652,11864,15620,16584,16909,18358,19451,21064,21608,21678,23022, 2890,23330, 1419, 4466, 5590,16862,17081,12481,15709,19148,23465, 8360, 7157,12586,20916,20922,14005,15959,20703,21383,19403,14618, 1423, 8591,11166,21855, 6719, 2384, 8730,22058, 2556, 4422,16305,13601,  335,22506, 6038,10243,13597,19664,20872, 7963,18562,20684, 8087,20983, 3578, 1345, 2019, 4270, 5416, 6707, 6775, 7827, 8188, 8955,10260,11860,13874,16028,16122,16873,20792,21279, 9074,21303,15457,16771,19619,12878, 6508, 6509, 3044,16751,21953,10984, 5021, 9856,14462,17989,19621, 7361,11500,17231, 2122, 2963,12618,17324,18502, 3560,18963, 4098, 4986, 5046,11428,11610,12610, 6163,11789,13789,16594,13580,15418,19630, 5196, 5197, 8098,10905,15213,19902,20090,22715, 9019, 2541, 3577, 1120,11454,13395,14099,15207,16158,21920,22924,14030, 1864, 2766, 3584, 5904, 9018,11511,21997,22002,22981, 4195, 6566,12893,16429,18766,18809,20366,20575,21350,21503, 3897,10067,12219,19873,  625,  889, 1391, 1441, 1889, 2260, 3341, 3552, 3776, 4943, 5015, 6726, 6762, 8707, 8818, 8959, 9022, 9080, 9223,10877,10903,11616,13952,13988,15255,15561,15634,16742,19227,19316,19865,20649,21522,21828,21850, 1372, 5079, 4830, 6518,13407, 8720, 9319,17177,  510, 3515, 8859,10224,11414,12411,17216,14747, 4785,16964,21826, 2375, 7288,12241,15420,18890,18961,19449,21066, 2525, 4131, 8640, 8831,11382,16839,16936,17700,18432,19618,21242,14152,11237,14797, 1395, 1428, 4672,16098,16316,19280,19654,20690,17792,17807,  356,  414,  511, 2917, 5462, 7966, 8946,10310,11591,11698,12352,17405,18116,18821,19339,21351, 5261, 3659, 8394,10653,11284,11548,11592,13357,16405,18666,20926,21504,22409,23026, 5438,16219,  595,  865, 1856, 3291, 4477, 4679, 5605, 5708, 6291, 6708, 6773, 7080, 9398, 9650,11124,11567,14269,14703,16227,16979,17316,18962,19067,19835,20522,22349,16143, 8364, 9827, 2362,13205, 7781,12752,13093, 6275, 4078, 9675,12751,19710,21486,18281,14915,15768,16671,16947,19722, 5533, 5174,10740,16650,17669,  190, 8049,11242,21695, 2865,16481,   58, 5914,15968, 4974,20521,12300,14681,21597,22854, 6018,17837,19850, 7229, 3985,20352, 1161,14635,19943,  152,11987, 2428, 7119,16876,21251,22774,  715, 8482,12912, 2843, 3343,13408,13759,16072,13582, 3703, 2738, 3244, 6754,13708,14217,16406,19075,19251,20923,23079,23281, 8456, 4792, 9697,22405,   76, 6567, 8403,13804,16164,19162,21307,22870,18886, 5303, 9737,16095,16974, 1228,13776,20682,23141,12465, 7796, 8159,13565,19525,20331,14391,20357, 9620, 2114, 1555, 1762, 4518, 6066,10013,14223,15144,15460,16447,16747,17701,17746,18343,19705,20826,22481, 1224, 5792,12287,16354,22558, 1319, 8557,17332,21416,  388, 3089, 6892, 9286,10722,12873,16399, 9767,22337,   47,  880, 1412, 2524, 5910, 6735, 6903, 7892, 8190, 8410, 8531, 8839, 9381, 9542,10200,11232,12797,13305,13592,14224,15737,15987,16428,17156,17648,18395,19193,19200,21823,23395, 1989,  580,10022, 2628, 4663, 4821, 6233,12811,16855))
# SELECT
#     n.admissionid,
#     n.itemid,
#     n.item,
#     n.value,
#     n.measuredat,
#     n.registeredby,
#     a.weightgroup,
#     a.heightgroup,
#     a.gender,
#     a.agegroup,
#     a.admittedat,
#     a.patientid,
#     a.dateofdeath,
#     CASE
#         WHEN n.registeredby NOT LIKE '%Systeem%' THEN TRUE
#         ELSE FALSE
#     END AS manual_entry,
#     (n.measuredat - a.admittedat)/(1000*60) AS time,
# FROM numericitems n
# LEFT JOIN admissions a ON
#      n.admissionid = a.admissionid
# WHERE n.itemid IN (
#         6840,   --Natrium
#         6642,   --ABP Mean Blood Pressure
#         6679,   --Niet invasieve bloeddruk gemiddeld
#         6810,   --Bicarbonate I
#         6819,   --Chloride II
#         6835,   --Kalium mmol/l
#         6836,   --Kreatinine µmol/l (erroneously documented as µmol)
#         8794,   --UrineCAD
#         8796,   --UrineSupraPubis
#         8798,   --UrineSpontaan
#         8800,   --UrineIncontinentie
#         8803,   --UrineUP
#         8843,   --ABP gemiddeld II
#         9924,   --Natrium (bloed)
#         9927,   --Kalium (bloed) mmol/l
#         9941,   --Kreatinine (bloed) µmol/l
#         9555,   --Natrium Astrup
#         9556,   --Kalium Astrup mmol/l
#         9558,   --Chloride I
#         9930,   --Chloride II
#         9992,   --Bicarbonate II
#         9993,   --Bicarbonate III
#         10284,  --Na (onv.ISE) (bloed)
#         10285,  --K (onv.ISE) (bloed) mmol/l
#         10743,  --Nefrodrain li Uit
#         10745,  --Nefrodrain re Uit
#         14216,  --KREAT enzym. (bloed) µmol/l
#         14413,  --Chloride IV
#         19921,  --UrineSplint Li
#         19922   --UrineSplint Re
#     )
#     AND
#     (n.admissionid IN (6720,17191,17961, 4310,13925,12231, 1888,11884, 480,13611, 332,  125, 9284, 8865, 9651,12007,12008,10357, 1121, 3784, 7859,10358,15320,17652,19043,20137,20722, 2164,22537, 1927, 3958, 9406,11308,16789,20980, 9467,  751, 1317, 9092,23540,  460,  512, 9927,11542,15224, 3658, 7620, 9020,13441,16369,19066,21996, 8401,17696,14984,19130,22849, 1375, 1965, 1980, 2754, 3326, 3458, 4109, 4812, 4940, 5769, 6471, 7003, 8383, 8573,10350,14155,14571,17341,18339,19082,19134,19615,19689,19847,20219,21083,21247,21857,11319,22262, 2954, 3568, 6195, 8830,15681,15808,17077,20596, 1402, 4311, 5649, 6982, 2994, 4639,18759,22387,10771,19898, 4523,  223, 2049, 4429, 6590, 7772, 9986,10764,11483,12977,15204,15718,17264,17655,22427,22658, 2300,12733,23276,  252, 1914, 2215, 3073, 3105, 3333, 5225, 5832, 6959, 7351, 8295, 9041, 9810,10016,11069,11462,12141,13060,14512,17190,19663,   20,  208,  467,  733,  884,  892, 1084, 1108, 1677, 1692, 1900, 2193, 2307, 2381, 2397, 2494, 2692, 2933, 2953, 2982, 2999, 3365, 3558, 3630, 3802, 3831, 3874, 3891, 3908, 4002, 4080, 4193, 4347, 5164, 5525, 5908, 5979, 6034, 6131, 6158, 6242, 6251, 6673, 6763, 6862, 7254, 7441, 7841, 8043, 8652, 8753, 8814, 8936, 9252, 9254, 9345, 9471, 9546, 9849, 9871,10154,10553,10794,11079,11259,11289,11345,11905,11912,12113,12413,12565,12592,12660,12672,12781,12850,13182,13309,13346,13385,13421,13571,13681,13744,13846,13957,14175,14246,14314,14468,14549,14573,15489,15490,15610,15695,15738,15809,16048,16144,16161,16275,16341,16449,16707,16937,17770,18060,18070,18165,18188,18316,18413,18462,18509,18513,18652,18660,18696,18728,19069,19117,19241,19732,19886,19897,20176,20263,20493,20697,20854,20861,20942,21270,21450,21616,21858,21891,21973,21989,22038,22476,22571,22631,22655,22709,22719,22852,22874,23001,23212, 1680, 2755, 2879, 5928, 6310,10060,13348,13440,16422,17848, 2462, 2265, 2435, 2583, 3546, 7246,13315,15673,16562,  939, 5564, 6481, 7409,11858,14493,15845,15938,17506,19477,22780,22890,23050, 2411, 3074, 3152, 3627, 4168, 4731, 5871, 6033, 6411, 6589, 8298, 9605,11539,11828,12305,13802,15037,16513,16561,17562,18199,19478, 3368, 7284, 7285, 8705,13073,22202,22639,20591,  644, 2984, 3220, 3376, 3426, 3549, 3949, 4633, 5906, 6214,16013,16384,16388,16843,17111,19346,23373,  740, 7650, 8469, 8648, 9506,12126,16506,17360,19482,13261,  564, 1614, 5219, 6765, 7277, 9564, 9703,12956,15381,20545,20630,13532, 9895,10914, 1482, 2838, 3980, 5470, 6848, 7251,11920,21068,  446,  858,  953, 1296, 2010, 2845, 3190, 3319, 4484, 5498, 5656, 5697, 5897, 6551, 7059, 7981, 8427,10178,10600,12118,14474,14775,16211,18657,18673,21164,21781,22682,23150, 5840, 7667,15871,22699,  421, 5851,10896,11682,13814,14433,15304,20399,20558,21325,22142, 1309, 2589,16360,19000,19279, 2935,  159,  186,  243,  559,  801,  879,  888, 1679, 1756, 2264, 2365, 2479, 2648, 2750, 2791, 3010, 3327, 3450, 3735, 4038, 4355, 5387, 5651, 5711, 5721, 5937, 5964, 6151, 6736, 7047, 7376, 7726, 7931, 7985, 8088, 8279, 8367, 8521, 8562, 8717, 9194, 9818, 9842,10074,10644,10744,11002,11300,11433,11524,11675,11799,11866,11989,12042,12290,12318,12634,12806,12853,12866,12990,13063,13074,13239,13885,13963,14291,14382,14539,14887,15366,15383,16087,16378,16470,16632,16818,16875,17013,17479,17664,17694,17830,17833,17840,17931,18028,18131,18294,18308,18668,18863,18940,19590,19625,20307,20364,21028,21361,21657,22056,22356,22618,22674,22893,23031,23173,23328,23335,23336,23468, 9899,13072, 7662, 6582,13730,21710, 2600, 6970, 9717,12159,13761,17261,21895,23166, 8749,15937, 9636,15981,21934, 1374, 6253, 6898,10344,12158,17785,20584, 7256, 9146, 8874,10539,11239,15455,16996,22000,22340, 6860,21161, 1827,   25, 3905, 5251, 5829, 6122, 6640, 7504,14940,17629,20868,  576,  867, 4711, 8197, 8442, 8929,10023,14931,15442,15531,17651,17853,18622,19315,19970,20985,21059,21287, 1274, 6288,19262, 1845,15672,17168, 2807,18573, 1669, 3780, 5950,10056,10665,10735,13095,16061,16776,17025,17919,18678,22453,22966, 6377,18949,23388, 3031, 5347, 9922,13246,13929,14485,15805,16625,18734,21963,   70,  344,  502,  789,  988, 1003, 1392, 1758, 1804, 2096, 2550, 2604, 3405, 3758, 4094, 4202, 4837, 5000, 5113, 5170, 5263, 5477, 5848, 6421, 6865, 7286, 7680, 7937, 8740, 8765, 9368, 9597, 9654, 9914,10065,10121,10146,10208,10304,10409,10623,10909,10947,11051,11594,11612,11686,11916,11959,11993,12524,12697,12719,12740,12851,12991,13360,13480,13502,13738,13796,13984,14044,14192,14359,14557,14620,14941,15059,15064,15512,15771,15852,15991,16041,16280,16442,16454,16676,16689,16995,17150,17167,17612,17627,17638,17691,18646,18989,19435,19752,19856,20014,20344,20823,21228,21478,21808,22245,22313,22494,22568,23165,23225, 3362, 8261,14338,16683,17619,18393, 1353, 5089, 9255,12756,17903,19144,23470, 1135, 3136, 8691,11143,14939,17321,18504,19866,22823,23201, 4241, 7602, 8259,11293,12574,20388,20618,21571, 2482,15675,17079,21770,22608,  647, 4358,11807,13589,14344,15232,23462,22884, 5120, 5187,16758,22600,22767,23545,  213,12640,21243,23243, 1691, 2231, 6234,12981,18401,18898,21796,  560, 2151, 3579, 3785, 4886, 6102, 7689, 7863, 7988,13645,14646,17115,20115,  441, 3295, 4879, 7800, 8864, 9245,13517,13944,16365,16452,17459,22070,22603, 6645,   11,  165,  229,  245, 1602, 1613, 1727, 2024, 2237, 2285, 2346, 2581, 3160, 3300, 3390, 3840, 3997, 4015, 4163, 4500, 4535, 4668, 4673, 5008, 5019, 5335, 5442, 5557, 5789, 6055, 6238, 7235, 7460, 7823, 8041, 8096, 8594, 9382,10193,10401,10414,10843,10953,11177,11421,11893,11991,12211,12437,12922,13174,13299,13471,13545,13714,13766,14184,14339,14423,14630,14656,14805,14858,15086,15347,15662,15666,15827,16534,16656,16711,17049,17391,17517,17866,17922,18289,18517,18576,18725,18820,19436,19708,19950,20081,20211,20225,20411,21685,21752,21822,21860,21880,22144,22173,22435,22529,22673,22911,23341, 2829,14960,18368,19064,19424,22004,11690,  218,  685, 1262, 8255,14064,20083, 4799, 5107, 7010,11192, 9490,18451,  146,  859, 2350, 2835, 3129, 4921, 5203, 5866, 6100, 6983, 6990, 7786, 8802, 9410, 9889,11071,11341,14134,14335,16340,16627,16911,17936,22368,22916,   94, 1332, 2465, 5011, 5728, 6638, 6833, 6916, 7406, 7531, 7897, 8076, 8961, 9576, 9704,10036,11384,11445,11566,12492,13459,13719,13978,14615,16246,16326,16495,18522,19701,19727,19821,19966,20430,20976,21188,22261, 2277, 6152,  997, 1987, 4085,15886,19533,20323,  620, 2777, 2907, 7308,13269,20928, 1635, 2082, 2562, 6926, 6966, 7151, 8348, 8363, 9275,10009,12460,16230,21195,21865,22286, 5054, 8232,12960,14307,14545,23000, 1797, 6442, 8460, 9352,11265,11444, 8374,11408,11223, 4186,21286, 1070, 1449, 2580, 3715, 5579, 7630, 7643, 9670, 9774,11299,11579,13539,15317,15422,17852,18768,19354,19430,19444,20725,21466,21733,21803,21835,22518,23214,13835,16435, 3710, 4023, 6931, 8121, 8249,10166,10899,11017,15229,20993,  643, 7104, 8793, 9119,12193,12311,16244,18489,   66, 1115, 1180, 2980, 3989, 4770, 4802, 6053, 7669, 8212,11283,12345,13260,14363,14533,15877,15932,17078,17152,17662,17897,18109,18905,19521,21868,23164,10586,18812,19818,13387,  133, 8556, 9388,11090, 1278,21856, 2301, 2849,14342,20311,21824,12415, 1755, 2214, 9239,21442,  666, 2658, 5218, 7060,12473,13890,14670,17695,19267,20457,20851, 8453,15817,17058,20643, 4594, 6094,14437,15299,15961,19471,20313, 3396, 3682, 5531,21258,17498, 9535,14074, 2028, 3483, 3042,16276, 7340, 4350, 5409,12130,15501,20909,14499,15292, 1106, 3984, 6454, 9291,10918,11227,11736,12551,17716,18119,19748,22817, 5777,18787,22611,22837,20982, 1367,  348, 2189, 3918, 5775, 6635, 8990, 9181,11007,11570,13124,14072,20890,22927,18265,22998, 3928,15017,21642,19041, 8158,13755,16490,19277, 4505,15567, 4602, 5627,10512,14216,15363, 4042, 4622, 5221,11668,17403,  270, 1219, 8118,17469,17983,21683,22779,17823,17860,16915,17729,22132, 7905,14418,22348, 5619,12965, 4177, 6415, 7564,10214,23302, 2325, 3876,15914,14487,12933,18723,21277, 6210, 6642, 9494,12579,13704,20562, 2084, 3521, 8136, 1455, 9076,14632,14798,16090,22466, 4178,19202,22757, 5622, 6941,21647, 8321,11665,17760,17726, 5827, 9217,21155, 1595, 2893, 3016, 3661, 3706, 4715, 6112, 6181, 8634, 9693,11354,12932,13198,13458,14548,14851,16590,16778,17002,17672,19565,21626,22718, 6895, 6958,14142,  265,17100,11753,10992,12874, 2044, 2888, 3509, 3843, 5623, 6919, 7506, 7522, 9900,10446,10482,12723,13011,13991,15394,15571,15647,16165,17125,18068,18756,19155,12319,12542,23062, 9509, 4944,18206, 9385,14113,  203, 1310, 2054, 2657, 3301, 5740, 6924, 7349, 7632, 9852,11250,11465,12346,15680,17831,19804,22831,22832,13518,20272, 4791,12961, 2559, 2634, 5725,21754, 5540, 7293, 8853, 9397,10382,18873,20720,21753,  204, 7616, 7423,  495, 6945, 8618,10253,10331,12688,12734,13066,18633,20102,21049,21213,21406,22353, 3882, 1661, 3981,15557,18906,21533,12303, 4246, 4632, 5300, 8395,14675,15978,18959, 2422,15488,10532,12899, 5696, 7067, 7899,17951,15245,  225, 4216,12509,12510, 5299, 1032, 1361, 6709,17499, 8660,  188,  492,  992, 1437, 1522, 3296, 4034, 4398, 5736, 6235, 6351, 6793, 7432, 7839, 8035, 9007,10510,11729,11756,12974,13243,13642,15278,15849,16160,16206,17635,18150,18155,18291,18643,19582,19738,19814,19844,20005,20933,21810,22478,22543,23420, 5757,19790,22031,17071,  273,  274, 1112, 1456, 1590, 1715, 1724, 1732, 1809, 1958, 2368, 2402, 2669, 5127, 8054, 8560, 8927, 9438, 9804,10658,11119,11359,13087,13200,13489,13879,14518,16015,16836,16837,16883,17856,18407,19052,19074,19388,20557,21742,22771,22830,23361,23482, 1149, 2840, 8196, 9929,12296,15248,18807,21038, 8423, 7682,  210,  514, 3966, 4854, 7214, 7444, 8110,13617,16093,17389,18721,21134,22995, 1546, 1947, 6683, 7529, 7996,18140,19323,22454,22860,  720, 5560, 7056,15127,20066, 4205,10635,18298, 1315, 6355,14078,  676,  970, 1946, 4405, 4576, 5152, 5334, 5584, 5692, 6320, 6887, 7198, 7205, 8007, 8320, 8494, 8921, 8926, 9370, 9479,10150,11270,12578,13692,13933,14171,14631,14695,15423,16557,16720,18008,18292,18612,20138,20755,20768,21760,21906,22221,22267,22627,22717,23363,23488, 2052, 3856,13977,23190, 2952, 5003, 5572,10021,11559,  918, 9154,11982,20667,20816,17183,22548,23406,  977, 3756, 8382, 8440, 8848, 8857, 9244, 9801, 9976,10500,10930,12263,13251,13528,14882,14963,15910,18677,18699,18913,19542,20174,22160,13170, 2540, 4210, 8519,  863, 1434, 3018, 3598, 3814, 4925, 5153, 5573, 5706, 5716, 5802, 7949, 8021, 9726, 9742,10119,11778,11837,12023,14181,14994,16152,17084,17395,18749,19051,20216,20255,20516,20677,22166,22845,23472, 3159, 6853, 8751,11142,13783, 6331,17550,15275,17390,16138,  917, 2108, 3019, 3823, 4142, 7363,10845,11797,19091, 1457, 1681, 3686, 6604,11247,11409,11486,15578,16267,20268,22566,23282,23458, 7534,10622,22552,  645, 8145,13824,16770,17463, 1429, 9193,19329,22009,15942,  442,  919, 2789, 3699, 6016, 6499,10145,10317,10552,12645,12685,16301,17052,20341,20437,21727,23483, 9148, 9634,16304,17467,20356,20704,16820,17387,23140, 3742,15605,19356,  299, 8500, 9779,12897,13880,13903, 4189,13228,15370,12284, 7641,12654, 7177,20838,11307, 3533,19288,22145, 2070, 4923,16914, 6936,11059, 1674, 1169, 4083, 4524, 5624,10445,10524,11235,18415,22668,23193,   99, 1660,15466,15881,16850,17074,21396,21959,22641,23514,  774, 3857, 4353, 4572, 5753, 7482, 7599, 7888, 8592, 9010, 9912,11832,12371,12661,14115,15548,15555,15558,16549,17243,19494,19583,19805,20517,20602,21755,23304,17170, 3974, 7305,17673,16111,21209, 1348,15597, 3893, 8729, 8974,11145, 7663, 9903,11495,15566,16668,21515,22238, 1260, 1403, 3058, 5873, 6261, 6409, 6803, 7877,10235,10951,13013,13784,16329,17594,19138,20355,20413,20476,21119,22455,22672,22698,14165,15660,16173,20574,22501, 7468, 8174,10080,10215,15921,19938,11823,11970,16701, 4859, 4992, 9886,10342,20250, 1252, 3202, 3323,11072,18133,22207,10087,12003,16217,18950,18951,20811,13481,  318,  479,  976, 1411, 1847, 2115, 3059, 3111, 5283, 6271, 8142, 8963, 8977, 9463, 9473, 9748,10217,10625,11768,11824,12839,13020,13190,14389,14392,14397,14758,15694,16730,17134,17242,17428,17489,17883,18127,18714,18954,18955,19372,19495,20231,20293,21022,21401,21827,23171,11898,13953, 4835,15518,18249, 8216,15191,15632, 2457,12258,12313,13626,17314,19334, 3520,12662, 6436, 8250, 1502, 2046, 2569, 3986, 7025, 8945,10929,12844,16070,16552,21172,12255, 5397, 9328, 9530,11013,13788,22721, 2099, 4906,19395,  877, 2821, 5128,13221, 8677,15717,  272,  310, 2779, 3112, 3563, 4627, 4691, 4855, 4991, 5803, 5952, 7431, 7558, 9841, 9910, 9968,11095,11205,11551,12310,13040,13128,14770,15253,15529,16759,18290,18421,18882,19224,20106,22169,22687,23128,23372, 1102, 1237, 2284, 2946, 4019, 4053, 5034, 5069, 6714, 6826, 7554, 7976,14086,15454,15867,16468,16624,19940,20088,20573,20964,21130,21487,21873,22239,22887,12298,16533,  134,  547,  584,  800, 1013, 1018, 1105, 1123, 1547, 1688, 1754, 2483, 2769, 3068, 3140, 3336, 3656, 3855, 3910, 4065, 4696, 5427, 5975, 6050, 6276, 6668, 6676, 7019, 7021, 7134, 7319, 7337, 7370, 7576, 7914, 8005, 8247, 8372, 8517, 8540, 9107, 9206, 9247, 9349, 9440, 9512, 9662,10042,10242,10647,11025,11201,11260,11496,11666,11917,12577,12619,13090,13367,13734,13762,14055,14130,14145,14193,14421,14621,14643,14649,14876,14918,15396,15424,15617,15645,16201,16827,16967,17153,17241,17558,17762,17975,17995,18221,18790,19008,19090,19690,20011,20199,20251,20482,20812,20953,21168,21292,21392,21548,21611,21927,22104,22222,22257,22467,22694,22919,22923,23211,23476,23535, 1172, 8631,11248,12292,14700,17222,20107,20887, 3097, 2093, 6914,17182, 1513, 7391,14059,18010,20622,20139,20623,23186, 4476,12243,14018, 5401,12770,14496,15158,19245,20425,  108, 2050, 2466, 5695, 6820, 6883, 7402, 7447, 7774, 7955, 9182,10058,11769,12341,12529,12760,13955,14552,15528,15556,15616,17297,18661,18715,21510,22848,  957, 4624, 5100, 7474, 4766, 6675,12758,14469,   44,  896, 1027, 1464, 3008, 3498, 4561, 5284,10348,13562,14257,16168,19629, 5905, 6223, 8001, 8815,10368,12972,23480, 5542, 1336, 8417,11761,16017,17723, 2606, 3720, 8690,12192,12530,13273,16623,19563,  493,11748,21580,14896, 7040,14647,16448,20132, 9136,10505,14427,16941,21140,11006,20817,  160, 2147, 3006, 3783, 4889, 6071, 6196, 7961, 8122, 8686, 9806,10006,10976,11240,11634,13653,13902,15117,15706,16009,16248,17218,17720,18444,18869,22273,23155,23188, 1311, 2445, 2820, 2989, 4446, 4781, 5913, 6121, 8575, 8930,10112,10549,11082,11757,12244,13950,15111,15439,15517,17566,18009,18031,18301,23461,16129,   50,   91,  904, 1246, 1465, 2041, 2148, 2266, 3481, 3616, 4842, 4908, 5108, 5574, 5894, 6370, 6844, 7417, 7969, 8569, 8687, 8688,10323,10461,10829,11340,11691,11723,11947,12707,12761,13220,13521,13667,13690,13867,14019,14396,14479,14490,15081,15159,15446,15513,16427,17480,18058,18302,18446,18834,19164,19857,21465,21489,22134,22184,18597, 7623,10124, 5779, 1824, 2833,17849, 8277, 2473, 9409, 6641,18447, 3428, 4970,10943,  600, 2795,22862,14457, 1655, 1656, 1775, 4743,11150, 3056,20819,  677, 1879, 4346,16132,19610,20583,13934,15912,19081,  775, 7875, 4409, 7116, 3816,13410,22278, 2136,17082,12920, 9824,12449, 2205, 7379,17034,19253, 8187, 8643,12963,13700,22097,20585, 2579, 1749, 2188, 5458, 6201, 9322,11216,17466, 6546, 3076,11882,18582,  364,17616, 2955,12448,12927,18523,18391,19526, 1739,14090,22950,18870,14141, 8764,10326,19366, 6950, 1713, 9204,12336,19383, 4313,10343,13826,16517,  144, 7103,10341,10937, 6710,10597,21460, 1289, 2125, 3079,10088,10106,12281,12353,14862,16007,16612,17371,18216,19062,19717,21894,23382,11351,18347, 5382,14292, 6120,10000, 1618, 2853, 4410, 5206, 5439,12259,15611,15945,16733,18205,21154,21414,23075, 8834,11365,  403, 2553, 4805, 5440, 6005, 7446, 8741,12221,12470,13660,16672,21259,22596,22898, 5355,11685, 3104,10865,21874,18825, 1665,15713,17997,20053,20117, 3055,19217,19450, 1530, 1852, 2366,18225,18226, 7142,  387,18050,21199,  246, 5780, 6046, 7833, 8429, 8971,10051,10677,11010,11163,12246,20795,22805,22901,11349,13622, 1053, 7191,10381, 4727,11942,  901,  902, 5373, 6179,10395,14810, 5177, 5638, 7317,11490,12823,13270,14346,17407, 5313,  424, 2061, 2146, 2698, 3959, 5504, 7473, 9652,11864,15620,16584,16909,18358,19451,21064,21608,21678,23022, 2890,23330, 1419, 4466, 5590,16862,17081,12481,15709,19148,23465, 8360, 7157,12586,20916,20922,14005,15959,20703,21383,19403,14618, 1423, 8591,11166,21855, 6719, 2384, 8730,22058, 2556, 4422,16305,13601,  335,22506, 6038,10243,13597,19664,20872, 7963,18562,20684, 8087,20983, 3578, 1345, 2019, 4270, 5416, 6707, 6775, 7827, 8188, 8955,10260,11860,13874,16028,16122,16873,20792,21279, 9074,21303,15457,16771,19619,12878, 6508, 6509, 3044,16751,21953,10984, 5021, 9856,14462,17989,19621, 7361,11500,17231, 2122, 2963,12618,17324,18502, 3560,18963, 4098, 4986, 5046,11428,11610,12610, 6163,11789,13789,16594,13580,15418,19630, 5196, 5197, 8098,10905,15213,19902,20090,22715, 9019, 2541, 3577, 1120,11454,13395,14099,15207,16158,21920,22924,14030, 1864, 2766, 3584, 5904, 9018,11511,21997,22002,22981, 4195, 6566,12893,16429,18766,18809,20366,20575,21350,21503, 3897,10067,12219,19873,  625,  889, 1391, 1441, 1889, 2260, 3341, 3552, 3776, 4943, 5015, 6726, 6762, 8707, 8818, 8959, 9022, 9080, 9223,10877,10903,11616,13952,13988,15255,15561,15634,16742,19227,19316,19865,20649,21522,21828,21850, 1372, 5079, 4830, 6518,13407, 8720, 9319,17177,  510, 3515, 8859,10224,11414,12411,17216,14747, 4785,16964,21826, 2375, 7288,12241,15420,18890,18961,19449,21066, 2525, 4131, 8640, 8831,11382,16839,16936,17700,18432,19618,21242,14152,11237,14797, 1395, 1428, 4672,16098,16316,19280,19654,20690,17792,17807,  356,  414,  511, 2917, 5462, 7966, 8946,10310,11591,11698,12352,17405,18116,18821,19339,21351, 5261, 3659, 8394,10653,11284,11548,11592,13357,16405,18666,20926,21504,22409,23026, 5438,16219,  595,  865, 1856, 3291, 4477, 4679, 5605, 5708, 6291, 6708, 6773, 7080, 9398, 9650,11124,11567,14269,14703,16227,16979,17316,18962,19067,19835,20522,22349,16143, 8364, 9827, 2362,13205, 7781,12752,13093, 6275, 4078, 9675,12751,19710,21486,18281,14915,15768,16671,16947,19722, 5533, 5174,10740,16650,17669,  190, 8049,11242,21695, 2865,16481,   58, 5914,15968, 4974,20521,12300,14681,21597,22854, 6018,17837,19850, 7229, 3985,20352, 1161,14635,19943,  152,11987, 2428, 7119,16876,21251,22774,  715, 8482,12912, 2843, 3343,13408,13759,16072,13582, 3703, 2738, 3244, 6754,13708,14217,16406,19075,19251,20923,23079,23281, 8456, 4792, 9697,22405,   76, 6567, 8403,13804,16164,19162,21307,22870,18886, 5303, 9737,16095,16974, 1228,13776,20682,23141,12465, 7796, 8159,13565,19525,20331,14391,20357, 9620, 2114, 1555, 1762, 4518, 6066,10013,14223,15144,15460,16447,16747,17701,17746,18343,19705,20826,22481, 1224, 5792,12287,16354,22558, 1319, 8557,17332,21416,  388, 3089, 6892, 9286,10722,12873,16399, 9767,22337,   47,  880, 1412, 2524, 5910, 6735, 6903, 7892, 8190, 8410, 8531, 8839, 9381, 9542,10200,11232,12797,13305,13592,14224,15737,15987,16428,17156,17648,18395,19193,19200,21823,23395, 1989,  580,10022, 2628, 4663, 4821, 6233,12811,16855)) 
#     AND n.measuredat <= 72*60*60*1000 AND n.measuredat >= 0 --72h*60m*60s*1000ms

In [5]:
# state_space_query.to_csv('nieuw_state_space.csv')

# del(state_space_query)
# gc.collect()

In [6]:
# %%bigquery action_space_query 
# WITH adm AS (
#   SELECT 
#     a.admissionid,
#     a.weightgroup,
#     a.heightgroup,
#     a.gender,
#     a.agegroup,
#     a.admittedat,
#     a.patientid,
#     a.dateofdeath
#   FROM admissions a
#   WHERE a.admissionid IN ( 6720,17191,17961, 4310,13925,12231, 1888,11884,  480,13611,  332,  125, 9284, 8865, 9651,12007,12008,10357, 1121, 3784, 7859,10358,15320,17652,19043,20137,20722, 2164,22537, 1927, 3958, 9406,11308,16789,20980, 9467,  751, 1317, 9092,23540,  460,  512, 9927,11542,15224, 3658, 7620, 9020,13441,16369,19066,21996, 8401,17696,14984,19130,22849, 1375, 1965, 1980, 2754, 3326, 3458, 4109, 4812, 4940, 5769, 6471, 7003, 8383, 8573,10350,14155,14571,17341,18339,19082,19134,19615,19689,19847,20219,21083,21247,21857,11319,22262, 2954, 3568, 6195, 8830,15681,15808,17077,20596, 1402, 4311, 5649, 6982, 2994, 4639,18759,22387,10771,19898, 4523,  223, 2049, 4429, 6590, 7772, 9986,10764,11483,12977,15204,15718,17264,17655,22427,22658, 2300,12733,23276,  252, 1914, 2215, 3073, 3105, 3333, 5225, 5832, 6959, 7351, 8295, 9041, 9810,10016,11069,11462,12141,13060,14512,17190,19663,   20,  208,  467,  733,  884,  892, 1084, 1108, 1677, 1692, 1900, 2193, 2307, 2381, 2397, 2494, 2692, 2933, 2953, 2982, 2999, 3365, 3558, 3630, 3802, 3831, 3874, 3891, 3908, 4002, 4080, 4193, 4347, 5164, 5525, 5908, 5979, 6034, 6131, 6158, 6242, 6251, 6673, 6763, 6862, 7254, 7441, 7841, 8043, 8652, 8753, 8814, 8936, 9252, 9254, 9345, 9471, 9546, 9849, 9871,10154,10553,10794,11079,11259,11289,11345,11905,11912,12113,12413,12565,12592,12660,12672,12781,12850,13182,13309,13346,13385,13421,13571,13681,13744,13846,13957,14175,14246,14314,14468,14549,14573,15489,15490,15610,15695,15738,15809,16048,16144,16161,16275,16341,16449,16707,16937,17770,18060,18070,18165,18188,18316,18413,18462,18509,18513,18652,18660,18696,18728,19069,19117,19241,19732,19886,19897,20176,20263,20493,20697,20854,20861,20942,21270,21450,21616,21858,21891,21973,21989,22038,22476,22571,22631,22655,22709,22719,22852,22874,23001,23212, 1680, 2755, 2879, 5928, 6310,10060,13348,13440,16422,17848, 2462, 2265, 2435, 2583, 3546, 7246,13315,15673,16562,  939, 5564, 6481, 7409,11858,14493,15845,15938,17506,19477,22780,22890,23050, 2411, 3074, 3152, 3627, 4168, 4731, 5871, 6033, 6411, 6589, 8298, 9605,11539,11828,12305,13802,15037,16513,16561,17562,18199,19478, 3368, 7284, 7285, 8705,13073,22202,22639,20591,  644, 2984, 3220, 3376, 3426, 3549, 3949, 4633, 5906, 6214,16013,16384,16388,16843,17111,19346,23373,  740, 7650, 8469, 8648, 9506,12126,16506,17360,19482,13261,  564, 1614, 5219, 6765, 7277, 9564, 9703,12956,15381,20545,20630,13532, 9895,10914, 1482, 2838, 3980, 5470, 6848, 7251,11920,21068,  446,  858,  953, 1296, 2010, 2845, 3190, 3319, 4484, 5498, 5656, 5697, 5897, 6551, 7059, 7981, 8427,10178,10600,12118,14474,14775,16211,18657,18673,21164,21781,22682,23150, 5840, 7667,15871,22699,  421, 5851,10896,11682,13814,14433,15304,20399,20558,21325,22142, 1309, 2589,16360,19000,19279, 2935,  159,  186,  243,  559,  801,  879,  888, 1679, 1756, 2264, 2365, 2479, 2648, 2750, 2791, 3010, 3327, 3450, 3735, 4038, 4355, 5387, 5651, 5711, 5721, 5937, 5964, 6151, 6736, 7047, 7376, 7726, 7931, 7985, 8088, 8279, 8367, 8521, 8562, 8717, 9194, 9818, 9842,10074,10644,10744,11002,11300,11433,11524,11675,11799,11866,11989,12042,12290,12318,12634,12806,12853,12866,12990,13063,13074,13239,13885,13963,14291,14382,14539,14887,15366,15383,16087,16378,16470,16632,16818,16875,17013,17479,17664,17694,17830,17833,17840,17931,18028,18131,18294,18308,18668,18863,18940,19590,19625,20307,20364,21028,21361,21657,22056,22356,22618,22674,22893,23031,23173,23328,23335,23336,23468, 9899,13072, 7662, 6582,13730,21710, 2600, 6970, 9717,12159,13761,17261,21895,23166, 8749,15937, 9636,15981,21934, 1374, 6253, 6898,10344,12158,17785,20584, 7256, 9146, 8874,10539,11239,15455,16996,22000,22340, 6860,21161, 1827,   25, 3905, 5251, 5829, 6122, 6640, 7504,14940,17629,20868,  576,  867, 4711, 8197, 8442, 8929,10023,14931,15442,15531,17651,17853,18622,19315,19970,20985,21059,21287, 1274, 6288,19262, 1845,15672,17168, 2807,18573, 1669, 3780, 5950,10056,10665,10735,13095,16061,16776,17025,17919,18678,22453,22966, 6377,18949,23388, 3031, 5347, 9922,13246,13929,14485,15805,16625,18734,21963,   70,  344,  502,  789,  988, 1003, 1392, 1758, 1804, 2096, 2550, 2604, 3405, 3758, 4094, 4202, 4837, 5000, 5113, 5170, 5263, 5477, 5848, 6421, 6865, 7286, 7680, 7937, 8740, 8765, 9368, 9597, 9654, 9914,10065,10121,10146,10208,10304,10409,10623,10909,10947,11051,11594,11612,11686,11916,11959,11993,12524,12697,12719,12740,12851,12991,13360,13480,13502,13738,13796,13984,14044,14192,14359,14557,14620,14941,15059,15064,15512,15771,15852,15991,16041,16280,16442,16454,16676,16689,16995,17150,17167,17612,17627,17638,17691,18646,18989,19435,19752,19856,20014,20344,20823,21228,21478,21808,22245,22313,22494,22568,23165,23225, 3362, 8261,14338,16683,17619,18393, 1353, 5089, 9255,12756,17903,19144,23470, 1135, 3136, 8691,11143,14939,17321,18504,19866,22823,23201, 4241, 7602, 8259,11293,12574,20388,20618,21571, 2482,15675,17079,21770,22608,  647, 4358,11807,13589,14344,15232,23462,22884, 5120, 5187,16758,22600,22767,23545,  213,12640,21243,23243, 1691, 2231, 6234,12981,18401,18898,21796,  560, 2151, 3579, 3785, 4886, 6102, 7689, 7863, 7988,13645,14646,17115,20115,  441, 3295, 4879, 7800, 8864, 9245,13517,13944,16365,16452,17459,22070,22603, 6645,   11,  165,  229,  245, 1602, 1613, 1727, 2024, 2237, 2285, 2346, 2581, 3160, 3300, 3390, 3840, 3997, 4015, 4163, 4500, 4535, 4668, 4673, 5008, 5019, 5335, 5442, 5557, 5789, 6055, 6238, 7235, 7460, 7823, 8041, 8096, 8594, 9382,10193,10401,10414,10843,10953,11177,11421,11893,11991,12211,12437,12922,13174,13299,13471,13545,13714,13766,14184,14339,14423,14630,14656,14805,14858,15086,15347,15662,15666,15827,16534,16656,16711,17049,17391,17517,17866,17922,18289,18517,18576,18725,18820,19436,19708,19950,20081,20211,20225,20411,21685,21752,21822,21860,21880,22144,22173,22435,22529,22673,22911,23341, 2829,14960,18368,19064,19424,22004,11690,  218,  685, 1262, 8255,14064,20083, 4799, 5107, 7010,11192, 9490,18451,  146,  859, 2350, 2835, 3129, 4921, 5203, 5866, 6100, 6983, 6990, 7786, 8802, 9410, 9889,11071,11341,14134,14335,16340,16627,16911,17936,22368,22916,   94, 1332, 2465, 5011, 5728, 6638, 6833, 6916, 7406, 7531, 7897, 8076, 8961, 9576, 9704,10036,11384,11445,11566,12492,13459,13719,13978,14615,16246,16326,16495,18522,19701,19727,19821,19966,20430,20976,21188,22261, 2277, 6152,  997, 1987, 4085,15886,19533,20323,  620, 2777, 2907, 7308,13269,20928, 1635, 2082, 2562, 6926, 6966, 7151, 8348, 8363, 9275,10009,12460,16230,21195,21865,22286, 5054, 8232,12960,14307,14545,23000, 1797, 6442, 8460, 9352,11265,11444, 8374,11408,11223, 4186,21286, 1070, 1449, 2580, 3715, 5579, 7630, 7643, 9670, 9774,11299,11579,13539,15317,15422,17852,18768,19354,19430,19444,20725,21466,21733,21803,21835,22518,23214,13835,16435, 3710, 4023, 6931, 8121, 8249,10166,10899,11017,15229,20993,  643, 7104, 8793, 9119,12193,12311,16244,18489,   66, 1115, 1180, 2980, 3989, 4770, 4802, 6053, 7669, 8212,11283,12345,13260,14363,14533,15877,15932,17078,17152,17662,17897,18109,18905,19521,21868,23164,10586,18812,19818,13387,  133, 8556, 9388,11090, 1278,21856, 2301, 2849,14342,20311,21824,12415, 1755, 2214, 9239,21442,  666, 2658, 5218, 7060,12473,13890,14670,17695,19267,20457,20851, 8453,15817,17058,20643, 4594, 6094,14437,15299,15961,19471,20313, 3396, 3682, 5531,21258,17498, 9535,14074, 2028, 3483, 3042,16276, 7340, 4350, 5409,12130,15501,20909,14499,15292, 1106, 3984, 6454, 9291,10918,11227,11736,12551,17716,18119,19748,22817, 5777,18787,22611,22837,20982, 1367,  348, 2189, 3918, 5775, 6635, 8990, 9181,11007,11570,13124,14072,20890,22927,18265,22998, 3928,15017,21642,19041, 8158,13755,16490,19277, 4505,15567, 4602, 5627,10512,14216,15363, 4042, 4622, 5221,11668,17403,  270, 1219, 8118,17469,17983,21683,22779,17823,17860,16915,17729,22132, 7905,14418,22348, 5619,12965, 4177, 6415, 7564,10214,23302, 2325, 3876,15914,14487,12933,18723,21277, 6210, 6642, 9494,12579,13704,20562, 2084, 3521, 8136, 1455, 9076,14632,14798,16090,22466, 4178,19202,22757, 5622, 6941,21647, 8321,11665,17760,17726, 5827, 9217,21155, 1595, 2893, 3016, 3661, 3706, 4715, 6112, 6181, 8634, 9693,11354,12932,13198,13458,14548,14851,16590,16778,17002,17672,19565,21626,22718, 6895, 6958,14142,  265,17100,11753,10992,12874, 2044, 2888, 3509, 3843, 5623, 6919, 7506, 7522, 9900,10446,10482,12723,13011,13991,15394,15571,15647,16165,17125,18068,18756,19155,12319,12542,23062, 9509, 4944,18206, 9385,14113,  203, 1310, 2054, 2657, 3301, 5740, 6924, 7349, 7632, 9852,11250,11465,12346,15680,17831,19804,22831,22832,13518,20272, 4791,12961, 2559, 2634, 5725,21754, 5540, 7293, 8853, 9397,10382,18873,20720,21753,  204, 7616, 7423,  495, 6945, 8618,10253,10331,12688,12734,13066,18633,20102,21049,21213,21406,22353, 3882, 1661, 3981,15557,18906,21533,12303, 4246, 4632, 5300, 8395,14675,15978,18959, 2422,15488,10532,12899, 5696, 7067, 7899,17951,15245,  225, 4216,12509,12510, 5299, 1032, 1361, 6709,17499, 8660,  188,  492,  992, 1437, 1522, 3296, 4034, 4398, 5736, 6235, 6351, 6793, 7432, 7839, 8035, 9007,10510,11729,11756,12974,13243,13642,15278,15849,16160,16206,17635,18150,18155,18291,18643,19582,19738,19814,19844,20005,20933,21810,22478,22543,23420, 5757,19790,22031,17071,  273,  274, 1112, 1456, 1590, 1715, 1724, 1732, 1809, 1958, 2368, 2402, 2669, 5127, 8054, 8560, 8927, 9438, 9804,10658,11119,11359,13087,13200,13489,13879,14518,16015,16836,16837,16883,17856,18407,19052,19074,19388,20557,21742,22771,22830,23361,23482, 1149, 2840, 8196, 9929,12296,15248,18807,21038, 8423, 7682,  210,  514, 3966, 4854, 7214, 7444, 8110,13617,16093,17389,18721,21134,22995, 1546, 1947, 6683, 7529, 7996,18140,19323,22454,22860,  720, 5560, 7056,15127,20066, 4205,10635,18298, 1315, 6355,14078,  676,  970, 1946, 4405, 4576, 5152, 5334, 5584, 5692, 6320, 6887, 7198, 7205, 8007, 8320, 8494, 8921, 8926, 9370, 9479,10150,11270,12578,13692,13933,14171,14631,14695,15423,16557,16720,18008,18292,18612,20138,20755,20768,21760,21906,22221,22267,22627,22717,23363,23488, 2052, 3856,13977,23190, 2952, 5003, 5572,10021,11559,  918, 9154,11982,20667,20816,17183,22548,23406,  977, 3756, 8382, 8440, 8848, 8857, 9244, 9801, 9976,10500,10930,12263,13251,13528,14882,14963,15910,18677,18699,18913,19542,20174,22160,13170, 2540, 4210, 8519,  863, 1434, 3018, 3598, 3814, 4925, 5153, 5573, 5706, 5716, 5802, 7949, 8021, 9726, 9742,10119,11778,11837,12023,14181,14994,16152,17084,17395,18749,19051,20216,20255,20516,20677,22166,22845,23472, 3159, 6853, 8751,11142,13783, 6331,17550,15275,17390,16138,  917, 2108, 3019, 3823, 4142, 7363,10845,11797,19091, 1457, 1681, 3686, 6604,11247,11409,11486,15578,16267,20268,22566,23282,23458, 7534,10622,22552,  645, 8145,13824,16770,17463, 1429, 9193,19329,22009,15942,  442,  919, 2789, 3699, 6016, 6499,10145,10317,10552,12645,12685,16301,17052,20341,20437,21727,23483, 9148, 9634,16304,17467,20356,20704,16820,17387,23140, 3742,15605,19356,  299, 8500, 9779,12897,13880,13903, 4189,13228,15370,12284, 7641,12654, 7177,20838,11307, 3533,19288,22145, 2070, 4923,16914, 6936,11059, 1674, 1169, 4083, 4524, 5624,10445,10524,11235,18415,22668,23193,   99, 1660,15466,15881,16850,17074,21396,21959,22641,23514,  774, 3857, 4353, 4572, 5753, 7482, 7599, 7888, 8592, 9010, 9912,11832,12371,12661,14115,15548,15555,15558,16549,17243,19494,19583,19805,20517,20602,21755,23304,17170, 3974, 7305,17673,16111,21209, 1348,15597, 3893, 8729, 8974,11145, 7663, 9903,11495,15566,16668,21515,22238, 1260, 1403, 3058, 5873, 6261, 6409, 6803, 7877,10235,10951,13013,13784,16329,17594,19138,20355,20413,20476,21119,22455,22672,22698,14165,15660,16173,20574,22501, 7468, 8174,10080,10215,15921,19938,11823,11970,16701, 4859, 4992, 9886,10342,20250, 1252, 3202, 3323,11072,18133,22207,10087,12003,16217,18950,18951,20811,13481,  318,  479,  976, 1411, 1847, 2115, 3059, 3111, 5283, 6271, 8142, 8963, 8977, 9463, 9473, 9748,10217,10625,11768,11824,12839,13020,13190,14389,14392,14397,14758,15694,16730,17134,17242,17428,17489,17883,18127,18714,18954,18955,19372,19495,20231,20293,21022,21401,21827,23171,11898,13953, 4835,15518,18249, 8216,15191,15632, 2457,12258,12313,13626,17314,19334, 3520,12662, 6436, 8250, 1502, 2046, 2569, 3986, 7025, 8945,10929,12844,16070,16552,21172,12255, 5397, 9328, 9530,11013,13788,22721, 2099, 4906,19395,  877, 2821, 5128,13221, 8677,15717,  272,  310, 2779, 3112, 3563, 4627, 4691, 4855, 4991, 5803, 5952, 7431, 7558, 9841, 9910, 9968,11095,11205,11551,12310,13040,13128,14770,15253,15529,16759,18290,18421,18882,19224,20106,22169,22687,23128,23372, 1102, 1237, 2284, 2946, 4019, 4053, 5034, 5069, 6714, 6826, 7554, 7976,14086,15454,15867,16468,16624,19940,20088,20573,20964,21130,21487,21873,22239,22887,12298,16533,  134,  547,  584,  800, 1013, 1018, 1105, 1123, 1547, 1688, 1754, 2483, 2769, 3068, 3140, 3336, 3656, 3855, 3910, 4065, 4696, 5427, 5975, 6050, 6276, 6668, 6676, 7019, 7021, 7134, 7319, 7337, 7370, 7576, 7914, 8005, 8247, 8372, 8517, 8540, 9107, 9206, 9247, 9349, 9440, 9512, 9662,10042,10242,10647,11025,11201,11260,11496,11666,11917,12577,12619,13090,13367,13734,13762,14055,14130,14145,14193,14421,14621,14643,14649,14876,14918,15396,15424,15617,15645,16201,16827,16967,17153,17241,17558,17762,17975,17995,18221,18790,19008,19090,19690,20011,20199,20251,20482,20812,20953,21168,21292,21392,21548,21611,21927,22104,22222,22257,22467,22694,22919,22923,23211,23476,23535, 1172, 8631,11248,12292,14700,17222,20107,20887, 3097, 2093, 6914,17182, 1513, 7391,14059,18010,20622,20139,20623,23186, 4476,12243,14018, 5401,12770,14496,15158,19245,20425,  108, 2050, 2466, 5695, 6820, 6883, 7402, 7447, 7774, 7955, 9182,10058,11769,12341,12529,12760,13955,14552,15528,15556,15616,17297,18661,18715,21510,22848,  957, 4624, 5100, 7474, 4766, 6675,12758,14469,   44,  896, 1027, 1464, 3008, 3498, 4561, 5284,10348,13562,14257,16168,19629, 5905, 6223, 8001, 8815,10368,12972,23480, 5542, 1336, 8417,11761,16017,17723, 2606, 3720, 8690,12192,12530,13273,16623,19563,  493,11748,21580,14896, 7040,14647,16448,20132, 9136,10505,14427,16941,21140,11006,20817,  160, 2147, 3006, 3783, 4889, 6071, 6196, 7961, 8122, 8686, 9806,10006,10976,11240,11634,13653,13902,15117,15706,16009,16248,17218,17720,18444,18869,22273,23155,23188, 1311, 2445, 2820, 2989, 4446, 4781, 5913, 6121, 8575, 8930,10112,10549,11082,11757,12244,13950,15111,15439,15517,17566,18009,18031,18301,23461,16129,   50,   91,  904, 1246, 1465, 2041, 2148, 2266, 3481, 3616, 4842, 4908, 5108, 5574, 5894, 6370, 6844, 7417, 7969, 8569, 8687, 8688,10323,10461,10829,11340,11691,11723,11947,12707,12761,13220,13521,13667,13690,13867,14019,14396,14479,14490,15081,15159,15446,15513,16427,17480,18058,18302,18446,18834,19164,19857,21465,21489,22134,22184,18597, 7623,10124, 5779, 1824, 2833,17849, 8277, 2473, 9409, 6641,18447, 3428, 4970,10943,  600, 2795,22862,14457, 1655, 1656, 1775, 4743,11150, 3056,20819,  677, 1879, 4346,16132,19610,20583,13934,15912,19081,  775, 7875, 4409, 7116, 3816,13410,22278, 2136,17082,12920, 9824,12449, 2205, 7379,17034,19253, 8187, 8643,12963,13700,22097,20585, 2579, 1749, 2188, 5458, 6201, 9322,11216,17466, 6546, 3076,11882,18582,  364,17616, 2955,12448,12927,18523,18391,19526, 1739,14090,22950,18870,14141, 8764,10326,19366, 6950, 1713, 9204,12336,19383, 4313,10343,13826,16517,  144, 7103,10341,10937, 6710,10597,21460, 1289, 2125, 3079,10088,10106,12281,12353,14862,16007,16612,17371,18216,19062,19717,21894,23382,11351,18347, 5382,14292, 6120,10000, 1618, 2853, 4410, 5206, 5439,12259,15611,15945,16733,18205,21154,21414,23075, 8834,11365,  403, 2553, 4805, 5440, 6005, 7446, 8741,12221,12470,13660,16672,21259,22596,22898, 5355,11685, 3104,10865,21874,18825, 1665,15713,17997,20053,20117, 3055,19217,19450, 1530, 1852, 2366,18225,18226, 7142,  387,18050,21199,  246, 5780, 6046, 7833, 8429, 8971,10051,10677,11010,11163,12246,20795,22805,22901,11349,13622, 1053, 7191,10381, 4727,11942,  901,  902, 5373, 6179,10395,14810, 5177, 5638, 7317,11490,12823,13270,14346,17407, 5313,  424, 2061, 2146, 2698, 3959, 5504, 7473, 9652,11864,15620,16584,16909,18358,19451,21064,21608,21678,23022, 2890,23330, 1419, 4466, 5590,16862,17081,12481,15709,19148,23465, 8360, 7157,12586,20916,20922,14005,15959,20703,21383,19403,14618, 1423, 8591,11166,21855, 6719, 2384, 8730,22058, 2556, 4422,16305,13601,  335,22506, 6038,10243,13597,19664,20872, 7963,18562,20684, 8087,20983, 3578, 1345, 2019, 4270, 5416, 6707, 6775, 7827, 8188, 8955,10260,11860,13874,16028,16122,16873,20792,21279, 9074,21303,15457,16771,19619,12878, 6508, 6509, 3044,16751,21953,10984, 5021, 9856,14462,17989,19621, 7361,11500,17231, 2122, 2963,12618,17324,18502, 3560,18963, 4098, 4986, 5046,11428,11610,12610, 6163,11789,13789,16594,13580,15418,19630, 5196, 5197, 8098,10905,15213,19902,20090,22715, 9019, 2541, 3577, 1120,11454,13395,14099,15207,16158,21920,22924,14030, 1864, 2766, 3584, 5904, 9018,11511,21997,22002,22981, 4195, 6566,12893,16429,18766,18809,20366,20575,21350,21503, 3897,10067,12219,19873,  625,  889, 1391, 1441, 1889, 2260, 3341, 3552, 3776, 4943, 5015, 6726, 6762, 8707, 8818, 8959, 9022, 9080, 9223,10877,10903,11616,13952,13988,15255,15561,15634,16742,19227,19316,19865,20649,21522,21828,21850, 1372, 5079, 4830, 6518,13407, 8720, 9319,17177,  510, 3515, 8859,10224,11414,12411,17216,14747, 4785,16964,21826, 2375, 7288,12241,15420,18890,18961,19449,21066, 2525, 4131, 8640, 8831,11382,16839,16936,17700,18432,19618,21242,14152,11237,14797, 1395, 1428, 4672,16098,16316,19280,19654,20690,17792,17807,  356,  414,  511, 2917, 5462, 7966, 8946,10310,11591,11698,12352,17405,18116,18821,19339,21351, 5261, 3659, 8394,10653,11284,11548,11592,13357,16405,18666,20926,21504,22409,23026, 5438,16219,  595,  865, 1856, 3291, 4477, 4679, 5605, 5708, 6291, 6708, 6773, 7080, 9398, 9650,11124,11567,14269,14703,16227,16979,17316,18962,19067,19835,20522,22349,16143, 8364, 9827, 2362,13205, 7781,12752,13093, 6275, 4078, 9675,12751,19710,21486,18281,14915,15768,16671,16947,19722, 5533, 5174,10740,16650,17669,  190, 8049,11242,21695, 2865,16481,   58, 5914,15968, 4974,20521,12300,14681,21597,22854, 6018,17837,19850, 7229, 3985,20352, 1161,14635,19943,  152,11987, 2428, 7119,16876,21251,22774,  715, 8482,12912, 2843, 3343,13408,13759,16072,13582, 3703, 2738, 3244, 6754,13708,14217,16406,19075,19251,20923,23079,23281, 8456, 4792, 9697,22405,   76, 6567, 8403,13804,16164,19162,21307,22870,18886, 5303, 9737,16095,16974, 1228,13776,20682,23141,12465, 7796, 8159,13565,19525,20331,14391,20357, 9620, 2114, 1555, 1762, 4518, 6066,10013,14223,15144,15460,16447,16747,17701,17746,18343,19705,20826,22481, 1224, 5792,12287,16354,22558, 1319, 8557,17332,21416,  388, 3089, 6892, 9286,10722,12873,16399, 9767,22337,   47,  880, 1412, 2524, 5910, 6735, 6903, 7892, 8190, 8410, 8531, 8839, 9381, 9542,10200,11232,12797,13305,13592,14224,15737,15987,16428,17156,17648,18395,19193,19200,21823,23395, 1989,  580,10022, 2628, 4663, 4821, 6233,12811,16855)
# ),
# fluid AS (
#   SELECT
#     d.admissionid,
#     d.ordercategory,
#     d.ordercategoryid,
#     (d.start - a.admittedat)/(1000*60) AS start_time,
#     (d.stop - a.admittedat)/(1000*60) AS stop_time,
#     d.fluidin
#   FROM drugitems d
#   LEFT JOIN admissions a ON
#     d.admissionid = a.admissionid
#   WHERE d.ordercategoryid NOT IN (29) AND d.start <= 72*60*60*1000 AND d.stop >= 0 --72h*60m*60s*1000ms 
#   AND (d.admissionid IN ( 6720,17191,17961, 4310,13925,12231, 1888,11884,  480,13611,  332,  125, 9284, 8865, 9651,12007,12008,10357, 1121, 3784, 7859,10358,15320,17652,19043,20137,20722, 2164,22537, 1927, 3958, 9406,11308,16789,20980, 9467,  751, 1317, 9092,23540,  460,  512, 9927,11542,15224, 3658, 7620, 9020,13441,16369,19066,21996, 8401,17696,14984,19130,22849, 1375, 1965, 1980, 2754, 3326, 3458, 4109, 4812, 4940, 5769, 6471, 7003, 8383, 8573,10350,14155,14571,17341,18339,19082,19134,19615,19689,19847,20219,21083,21247,21857,11319,22262, 2954, 3568, 6195, 8830,15681,15808,17077,20596, 1402, 4311, 5649, 6982, 2994, 4639,18759,22387,10771,19898, 4523,  223, 2049, 4429, 6590, 7772, 9986,10764,11483,12977,15204,15718,17264,17655,22427,22658, 2300,12733,23276,  252, 1914, 2215, 3073, 3105, 3333, 5225, 5832, 6959, 7351, 8295, 9041, 9810,10016,11069,11462,12141,13060,14512,17190,19663,   20,  208,  467,  733,  884,  892, 1084, 1108, 1677, 1692, 1900, 2193, 2307, 2381, 2397, 2494, 2692, 2933, 2953, 2982, 2999, 3365, 3558, 3630, 3802, 3831, 3874, 3891, 3908, 4002, 4080, 4193, 4347, 5164, 5525, 5908, 5979, 6034, 6131, 6158, 6242, 6251, 6673, 6763, 6862, 7254, 7441, 7841, 8043, 8652, 8753, 8814, 8936, 9252, 9254, 9345, 9471, 9546, 9849, 9871,10154,10553,10794,11079,11259,11289,11345,11905,11912,12113,12413,12565,12592,12660,12672,12781,12850,13182,13309,13346,13385,13421,13571,13681,13744,13846,13957,14175,14246,14314,14468,14549,14573,15489,15490,15610,15695,15738,15809,16048,16144,16161,16275,16341,16449,16707,16937,17770,18060,18070,18165,18188,18316,18413,18462,18509,18513,18652,18660,18696,18728,19069,19117,19241,19732,19886,19897,20176,20263,20493,20697,20854,20861,20942,21270,21450,21616,21858,21891,21973,21989,22038,22476,22571,22631,22655,22709,22719,22852,22874,23001,23212, 1680, 2755, 2879, 5928, 6310,10060,13348,13440,16422,17848, 2462, 2265, 2435, 2583, 3546, 7246,13315,15673,16562,  939, 5564, 6481, 7409,11858,14493,15845,15938,17506,19477,22780,22890,23050, 2411, 3074, 3152, 3627, 4168, 4731, 5871, 6033, 6411, 6589, 8298, 9605,11539,11828,12305,13802,15037,16513,16561,17562,18199,19478, 3368, 7284, 7285, 8705,13073,22202,22639,20591,  644, 2984, 3220, 3376, 3426, 3549, 3949, 4633, 5906, 6214,16013,16384,16388,16843,17111,19346,23373,  740, 7650, 8469, 8648, 9506,12126,16506,17360,19482,13261,  564, 1614, 5219, 6765, 7277, 9564, 9703,12956,15381,20545,20630,13532, 9895,10914, 1482, 2838, 3980, 5470, 6848, 7251,11920,21068,  446,  858,  953, 1296, 2010, 2845, 3190, 3319, 4484, 5498, 5656, 5697, 5897, 6551, 7059, 7981, 8427,10178,10600,12118,14474,14775,16211,18657,18673,21164,21781,22682,23150, 5840, 7667,15871,22699,  421, 5851,10896,11682,13814,14433,15304,20399,20558,21325,22142, 1309, 2589,16360,19000,19279, 2935,  159,  186,  243,  559,  801,  879,  888, 1679, 1756, 2264, 2365, 2479, 2648, 2750, 2791, 3010, 3327, 3450, 3735, 4038, 4355, 5387, 5651, 5711, 5721, 5937, 5964, 6151, 6736, 7047, 7376, 7726, 7931, 7985, 8088, 8279, 8367, 8521, 8562, 8717, 9194, 9818, 9842,10074,10644,10744,11002,11300,11433,11524,11675,11799,11866,11989,12042,12290,12318,12634,12806,12853,12866,12990,13063,13074,13239,13885,13963,14291,14382,14539,14887,15366,15383,16087,16378,16470,16632,16818,16875,17013,17479,17664,17694,17830,17833,17840,17931,18028,18131,18294,18308,18668,18863,18940,19590,19625,20307,20364,21028,21361,21657,22056,22356,22618,22674,22893,23031,23173,23328,23335,23336,23468, 9899,13072, 7662, 6582,13730,21710, 2600, 6970, 9717,12159,13761,17261,21895,23166, 8749,15937, 9636,15981,21934, 1374, 6253, 6898,10344,12158,17785,20584, 7256, 9146, 8874,10539,11239,15455,16996,22000,22340, 6860,21161, 1827,   25, 3905, 5251, 5829, 6122, 6640, 7504,14940,17629,20868,  576,  867, 4711, 8197, 8442, 8929,10023,14931,15442,15531,17651,17853,18622,19315,19970,20985,21059,21287, 1274, 6288,19262, 1845,15672,17168, 2807,18573, 1669, 3780, 5950,10056,10665,10735,13095,16061,16776,17025,17919,18678,22453,22966, 6377,18949,23388, 3031, 5347, 9922,13246,13929,14485,15805,16625,18734,21963,   70,  344,  502,  789,  988, 1003, 1392, 1758, 1804, 2096, 2550, 2604, 3405, 3758, 4094, 4202, 4837, 5000, 5113, 5170, 5263, 5477, 5848, 6421, 6865, 7286, 7680, 7937, 8740, 8765, 9368, 9597, 9654, 9914,10065,10121,10146,10208,10304,10409,10623,10909,10947,11051,11594,11612,11686,11916,11959,11993,12524,12697,12719,12740,12851,12991,13360,13480,13502,13738,13796,13984,14044,14192,14359,14557,14620,14941,15059,15064,15512,15771,15852,15991,16041,16280,16442,16454,16676,16689,16995,17150,17167,17612,17627,17638,17691,18646,18989,19435,19752,19856,20014,20344,20823,21228,21478,21808,22245,22313,22494,22568,23165,23225, 3362, 8261,14338,16683,17619,18393, 1353, 5089, 9255,12756,17903,19144,23470, 1135, 3136, 8691,11143,14939,17321,18504,19866,22823,23201, 4241, 7602, 8259,11293,12574,20388,20618,21571, 2482,15675,17079,21770,22608,  647, 4358,11807,13589,14344,15232,23462,22884, 5120, 5187,16758,22600,22767,23545,  213,12640,21243,23243, 1691, 2231, 6234,12981,18401,18898,21796,  560, 2151, 3579, 3785, 4886, 6102, 7689, 7863, 7988,13645,14646,17115,20115,  441, 3295, 4879, 7800, 8864, 9245,13517,13944,16365,16452,17459,22070,22603, 6645,   11,  165,  229,  245, 1602, 1613, 1727, 2024, 2237, 2285, 2346, 2581, 3160, 3300, 3390, 3840, 3997, 4015, 4163, 4500, 4535, 4668, 4673, 5008, 5019, 5335, 5442, 5557, 5789, 6055, 6238, 7235, 7460, 7823, 8041, 8096, 8594, 9382,10193,10401,10414,10843,10953,11177,11421,11893,11991,12211,12437,12922,13174,13299,13471,13545,13714,13766,14184,14339,14423,14630,14656,14805,14858,15086,15347,15662,15666,15827,16534,16656,16711,17049,17391,17517,17866,17922,18289,18517,18576,18725,18820,19436,19708,19950,20081,20211,20225,20411,21685,21752,21822,21860,21880,22144,22173,22435,22529,22673,22911,23341, 2829,14960,18368,19064,19424,22004,11690,  218,  685, 1262, 8255,14064,20083, 4799, 5107, 7010,11192, 9490,18451,  146,  859, 2350, 2835, 3129, 4921, 5203, 5866, 6100, 6983, 6990, 7786, 8802, 9410, 9889,11071,11341,14134,14335,16340,16627,16911,17936,22368,22916,   94, 1332, 2465, 5011, 5728, 6638, 6833, 6916, 7406, 7531, 7897, 8076, 8961, 9576, 9704,10036,11384,11445,11566,12492,13459,13719,13978,14615,16246,16326,16495,18522,19701,19727,19821,19966,20430,20976,21188,22261, 2277, 6152,  997, 1987, 4085,15886,19533,20323,  620, 2777, 2907, 7308,13269,20928, 1635, 2082, 2562, 6926, 6966, 7151, 8348, 8363, 9275,10009,12460,16230,21195,21865,22286, 5054, 8232,12960,14307,14545,23000, 1797, 6442, 8460, 9352,11265,11444, 8374,11408,11223, 4186,21286, 1070, 1449, 2580, 3715, 5579, 7630, 7643, 9670, 9774,11299,11579,13539,15317,15422,17852,18768,19354,19430,19444,20725,21466,21733,21803,21835,22518,23214,13835,16435, 3710, 4023, 6931, 8121, 8249,10166,10899,11017,15229,20993,  643, 7104, 8793, 9119,12193,12311,16244,18489,   66, 1115, 1180, 2980, 3989, 4770, 4802, 6053, 7669, 8212,11283,12345,13260,14363,14533,15877,15932,17078,17152,17662,17897,18109,18905,19521,21868,23164,10586,18812,19818,13387,  133, 8556, 9388,11090, 1278,21856, 2301, 2849,14342,20311,21824,12415, 1755, 2214, 9239,21442,  666, 2658, 5218, 7060,12473,13890,14670,17695,19267,20457,20851, 8453,15817,17058,20643, 4594, 6094,14437,15299,15961,19471,20313, 3396, 3682, 5531,21258,17498, 9535,14074, 2028, 3483, 3042,16276, 7340, 4350, 5409,12130,15501,20909,14499,15292, 1106, 3984, 6454, 9291,10918,11227,11736,12551,17716,18119,19748,22817, 5777,18787,22611,22837,20982, 1367,  348, 2189, 3918, 5775, 6635, 8990, 9181,11007,11570,13124,14072,20890,22927,18265,22998, 3928,15017,21642,19041, 8158,13755,16490,19277, 4505,15567, 4602, 5627,10512,14216,15363, 4042, 4622, 5221,11668,17403,  270, 1219, 8118,17469,17983,21683,22779,17823,17860,16915,17729,22132, 7905,14418,22348, 5619,12965, 4177, 6415, 7564,10214,23302, 2325, 3876,15914,14487,12933,18723,21277, 6210, 6642, 9494,12579,13704,20562, 2084, 3521, 8136, 1455, 9076,14632,14798,16090,22466, 4178,19202,22757, 5622, 6941,21647, 8321,11665,17760,17726, 5827, 9217,21155, 1595, 2893, 3016, 3661, 3706, 4715, 6112, 6181, 8634, 9693,11354,12932,13198,13458,14548,14851,16590,16778,17002,17672,19565,21626,22718, 6895, 6958,14142,  265,17100,11753,10992,12874, 2044, 2888, 3509, 3843, 5623, 6919, 7506, 7522, 9900,10446,10482,12723,13011,13991,15394,15571,15647,16165,17125,18068,18756,19155,12319,12542,23062, 9509, 4944,18206, 9385,14113,  203, 1310, 2054, 2657, 3301, 5740, 6924, 7349, 7632, 9852,11250,11465,12346,15680,17831,19804,22831,22832,13518,20272, 4791,12961, 2559, 2634, 5725,21754, 5540, 7293, 8853, 9397,10382,18873,20720,21753,  204, 7616, 7423,  495, 6945, 8618,10253,10331,12688,12734,13066,18633,20102,21049,21213,21406,22353, 3882, 1661, 3981,15557,18906,21533,12303, 4246, 4632, 5300, 8395,14675,15978,18959, 2422,15488,10532,12899, 5696, 7067, 7899,17951,15245,  225, 4216,12509,12510, 5299, 1032, 1361, 6709,17499, 8660,  188,  492,  992, 1437, 1522, 3296, 4034, 4398, 5736, 6235, 6351, 6793, 7432, 7839, 8035, 9007,10510,11729,11756,12974,13243,13642,15278,15849,16160,16206,17635,18150,18155,18291,18643,19582,19738,19814,19844,20005,20933,21810,22478,22543,23420, 5757,19790,22031,17071,  273,  274, 1112, 1456, 1590, 1715, 1724, 1732, 1809, 1958, 2368, 2402, 2669, 5127, 8054, 8560, 8927, 9438, 9804,10658,11119,11359,13087,13200,13489,13879,14518,16015,16836,16837,16883,17856,18407,19052,19074,19388,20557,21742,22771,22830,23361,23482, 1149, 2840, 8196, 9929,12296,15248,18807,21038, 8423, 7682,  210,  514, 3966, 4854, 7214, 7444, 8110,13617,16093,17389,18721,21134,22995, 1546, 1947, 6683, 7529, 7996,18140,19323,22454,22860,  720, 5560, 7056,15127,20066, 4205,10635,18298, 1315, 6355,14078,  676,  970, 1946, 4405, 4576, 5152, 5334, 5584, 5692, 6320, 6887, 7198, 7205, 8007, 8320, 8494, 8921, 8926, 9370, 9479,10150,11270,12578,13692,13933,14171,14631,14695,15423,16557,16720,18008,18292,18612,20138,20755,20768,21760,21906,22221,22267,22627,22717,23363,23488, 2052, 3856,13977,23190, 2952, 5003, 5572,10021,11559,  918, 9154,11982,20667,20816,17183,22548,23406,  977, 3756, 8382, 8440, 8848, 8857, 9244, 9801, 9976,10500,10930,12263,13251,13528,14882,14963,15910,18677,18699,18913,19542,20174,22160,13170, 2540, 4210, 8519,  863, 1434, 3018, 3598, 3814, 4925, 5153, 5573, 5706, 5716, 5802, 7949, 8021, 9726, 9742,10119,11778,11837,12023,14181,14994,16152,17084,17395,18749,19051,20216,20255,20516,20677,22166,22845,23472, 3159, 6853, 8751,11142,13783, 6331,17550,15275,17390,16138,  917, 2108, 3019, 3823, 4142, 7363,10845,11797,19091, 1457, 1681, 3686, 6604,11247,11409,11486,15578,16267,20268,22566,23282,23458, 7534,10622,22552,  645, 8145,13824,16770,17463, 1429, 9193,19329,22009,15942,  442,  919, 2789, 3699, 6016, 6499,10145,10317,10552,12645,12685,16301,17052,20341,20437,21727,23483, 9148, 9634,16304,17467,20356,20704,16820,17387,23140, 3742,15605,19356,  299, 8500, 9779,12897,13880,13903, 4189,13228,15370,12284, 7641,12654, 7177,20838,11307, 3533,19288,22145, 2070, 4923,16914, 6936,11059, 1674, 1169, 4083, 4524, 5624,10445,10524,11235,18415,22668,23193,   99, 1660,15466,15881,16850,17074,21396,21959,22641,23514,  774, 3857, 4353, 4572, 5753, 7482, 7599, 7888, 8592, 9010, 9912,11832,12371,12661,14115,15548,15555,15558,16549,17243,19494,19583,19805,20517,20602,21755,23304,17170, 3974, 7305,17673,16111,21209, 1348,15597, 3893, 8729, 8974,11145, 7663, 9903,11495,15566,16668,21515,22238, 1260, 1403, 3058, 5873, 6261, 6409, 6803, 7877,10235,10951,13013,13784,16329,17594,19138,20355,20413,20476,21119,22455,22672,22698,14165,15660,16173,20574,22501, 7468, 8174,10080,10215,15921,19938,11823,11970,16701, 4859, 4992, 9886,10342,20250, 1252, 3202, 3323,11072,18133,22207,10087,12003,16217,18950,18951,20811,13481,  318,  479,  976, 1411, 1847, 2115, 3059, 3111, 5283, 6271, 8142, 8963, 8977, 9463, 9473, 9748,10217,10625,11768,11824,12839,13020,13190,14389,14392,14397,14758,15694,16730,17134,17242,17428,17489,17883,18127,18714,18954,18955,19372,19495,20231,20293,21022,21401,21827,23171,11898,13953, 4835,15518,18249, 8216,15191,15632, 2457,12258,12313,13626,17314,19334, 3520,12662, 6436, 8250, 1502, 2046, 2569, 3986, 7025, 8945,10929,12844,16070,16552,21172,12255, 5397, 9328, 9530,11013,13788,22721, 2099, 4906,19395,  877, 2821, 5128,13221, 8677,15717,  272,  310, 2779, 3112, 3563, 4627, 4691, 4855, 4991, 5803, 5952, 7431, 7558, 9841, 9910, 9968,11095,11205,11551,12310,13040,13128,14770,15253,15529,16759,18290,18421,18882,19224,20106,22169,22687,23128,23372, 1102, 1237, 2284, 2946, 4019, 4053, 5034, 5069, 6714, 6826, 7554, 7976,14086,15454,15867,16468,16624,19940,20088,20573,20964,21130,21487,21873,22239,22887,12298,16533,  134,  547,  584,  800, 1013, 1018, 1105, 1123, 1547, 1688, 1754, 2483, 2769, 3068, 3140, 3336, 3656, 3855, 3910, 4065, 4696, 5427, 5975, 6050, 6276, 6668, 6676, 7019, 7021, 7134, 7319, 7337, 7370, 7576, 7914, 8005, 8247, 8372, 8517, 8540, 9107, 9206, 9247, 9349, 9440, 9512, 9662,10042,10242,10647,11025,11201,11260,11496,11666,11917,12577,12619,13090,13367,13734,13762,14055,14130,14145,14193,14421,14621,14643,14649,14876,14918,15396,15424,15617,15645,16201,16827,16967,17153,17241,17558,17762,17975,17995,18221,18790,19008,19090,19690,20011,20199,20251,20482,20812,20953,21168,21292,21392,21548,21611,21927,22104,22222,22257,22467,22694,22919,22923,23211,23476,23535, 1172, 8631,11248,12292,14700,17222,20107,20887, 3097, 2093, 6914,17182, 1513, 7391,14059,18010,20622,20139,20623,23186, 4476,12243,14018, 5401,12770,14496,15158,19245,20425,  108, 2050, 2466, 5695, 6820, 6883, 7402, 7447, 7774, 7955, 9182,10058,11769,12341,12529,12760,13955,14552,15528,15556,15616,17297,18661,18715,21510,22848,  957, 4624, 5100, 7474, 4766, 6675,12758,14469,   44,  896, 1027, 1464, 3008, 3498, 4561, 5284,10348,13562,14257,16168,19629, 5905, 6223, 8001, 8815,10368,12972,23480, 5542, 1336, 8417,11761,16017,17723, 2606, 3720, 8690,12192,12530,13273,16623,19563,  493,11748,21580,14896, 7040,14647,16448,20132, 9136,10505,14427,16941,21140,11006,20817,  160, 2147, 3006, 3783, 4889, 6071, 6196, 7961, 8122, 8686, 9806,10006,10976,11240,11634,13653,13902,15117,15706,16009,16248,17218,17720,18444,18869,22273,23155,23188, 1311, 2445, 2820, 2989, 4446, 4781, 5913, 6121, 8575, 8930,10112,10549,11082,11757,12244,13950,15111,15439,15517,17566,18009,18031,18301,23461,16129,   50,   91,  904, 1246, 1465, 2041, 2148, 2266, 3481, 3616, 4842, 4908, 5108, 5574, 5894, 6370, 6844, 7417, 7969, 8569, 8687, 8688,10323,10461,10829,11340,11691,11723,11947,12707,12761,13220,13521,13667,13690,13867,14019,14396,14479,14490,15081,15159,15446,15513,16427,17480,18058,18302,18446,18834,19164,19857,21465,21489,22134,22184,18597, 7623,10124, 5779, 1824, 2833,17849, 8277, 2473, 9409, 6641,18447, 3428, 4970,10943,  600, 2795,22862,14457, 1655, 1656, 1775, 4743,11150, 3056,20819,  677, 1879, 4346,16132,19610,20583,13934,15912,19081,  775, 7875, 4409, 7116, 3816,13410,22278, 2136,17082,12920, 9824,12449, 2205, 7379,17034,19253, 8187, 8643,12963,13700,22097,20585, 2579, 1749, 2188, 5458, 6201, 9322,11216,17466, 6546, 3076,11882,18582,  364,17616, 2955,12448,12927,18523,18391,19526, 1739,14090,22950,18870,14141, 8764,10326,19366, 6950, 1713, 9204,12336,19383, 4313,10343,13826,16517,  144, 7103,10341,10937, 6710,10597,21460, 1289, 2125, 3079,10088,10106,12281,12353,14862,16007,16612,17371,18216,19062,19717,21894,23382,11351,18347, 5382,14292, 6120,10000, 1618, 2853, 4410, 5206, 5439,12259,15611,15945,16733,18205,21154,21414,23075, 8834,11365,  403, 2553, 4805, 5440, 6005, 7446, 8741,12221,12470,13660,16672,21259,22596,22898, 5355,11685, 3104,10865,21874,18825, 1665,15713,17997,20053,20117, 3055,19217,19450, 1530, 1852, 2366,18225,18226, 7142,  387,18050,21199,  246, 5780, 6046, 7833, 8429, 8971,10051,10677,11010,11163,12246,20795,22805,22901,11349,13622, 1053, 7191,10381, 4727,11942,  901,  902, 5373, 6179,10395,14810, 5177, 5638, 7317,11490,12823,13270,14346,17407, 5313,  424, 2061, 2146, 2698, 3959, 5504, 7473, 9652,11864,15620,16584,16909,18358,19451,21064,21608,21678,23022, 2890,23330, 1419, 4466, 5590,16862,17081,12481,15709,19148,23465, 8360, 7157,12586,20916,20922,14005,15959,20703,21383,19403,14618, 1423, 8591,11166,21855, 6719, 2384, 8730,22058, 2556, 4422,16305,13601,  335,22506, 6038,10243,13597,19664,20872, 7963,18562,20684, 8087,20983, 3578, 1345, 2019, 4270, 5416, 6707, 6775, 7827, 8188, 8955,10260,11860,13874,16028,16122,16873,20792,21279, 9074,21303,15457,16771,19619,12878, 6508, 6509, 3044,16751,21953,10984, 5021, 9856,14462,17989,19621, 7361,11500,17231, 2122, 2963,12618,17324,18502, 3560,18963, 4098, 4986, 5046,11428,11610,12610, 6163,11789,13789,16594,13580,15418,19630, 5196, 5197, 8098,10905,15213,19902,20090,22715, 9019, 2541, 3577, 1120,11454,13395,14099,15207,16158,21920,22924,14030, 1864, 2766, 3584, 5904, 9018,11511,21997,22002,22981, 4195, 6566,12893,16429,18766,18809,20366,20575,21350,21503, 3897,10067,12219,19873,  625,  889, 1391, 1441, 1889, 2260, 3341, 3552, 3776, 4943, 5015, 6726, 6762, 8707, 8818, 8959, 9022, 9080, 9223,10877,10903,11616,13952,13988,15255,15561,15634,16742,19227,19316,19865,20649,21522,21828,21850, 1372, 5079, 4830, 6518,13407, 8720, 9319,17177,  510, 3515, 8859,10224,11414,12411,17216,14747, 4785,16964,21826, 2375, 7288,12241,15420,18890,18961,19449,21066, 2525, 4131, 8640, 8831,11382,16839,16936,17700,18432,19618,21242,14152,11237,14797, 1395, 1428, 4672,16098,16316,19280,19654,20690,17792,17807,  356,  414,  511, 2917, 5462, 7966, 8946,10310,11591,11698,12352,17405,18116,18821,19339,21351, 5261, 3659, 8394,10653,11284,11548,11592,13357,16405,18666,20926,21504,22409,23026, 5438,16219,  595,  865, 1856, 3291, 4477, 4679, 5605, 5708, 6291, 6708, 6773, 7080, 9398, 9650,11124,11567,14269,14703,16227,16979,17316,18962,19067,19835,20522,22349,16143, 8364, 9827, 2362,13205, 7781,12752,13093, 6275, 4078, 9675,12751,19710,21486,18281,14915,15768,16671,16947,19722, 5533, 5174,10740,16650,17669,  190, 8049,11242,21695, 2865,16481,   58, 5914,15968, 4974,20521,12300,14681,21597,22854, 6018,17837,19850, 7229, 3985,20352, 1161,14635,19943,  152,11987, 2428, 7119,16876,21251,22774,  715, 8482,12912, 2843, 3343,13408,13759,16072,13582, 3703, 2738, 3244, 6754,13708,14217,16406,19075,19251,20923,23079,23281, 8456, 4792, 9697,22405,   76, 6567, 8403,13804,16164,19162,21307,22870,18886, 5303, 9737,16095,16974, 1228,13776,20682,23141,12465, 7796, 8159,13565,19525,20331,14391,20357, 9620, 2114, 1555, 1762, 4518, 6066,10013,14223,15144,15460,16447,16747,17701,17746,18343,19705,20826,22481, 1224, 5792,12287,16354,22558, 1319, 8557,17332,21416,  388, 3089, 6892, 9286,10722,12873,16399, 9767,22337,   47,  880, 1412, 2524, 5910, 6735, 6903, 7892, 8190, 8410, 8531, 8839, 9381, 9542,10200,11232,12797,13305,13592,14224,15737,15987,16428,17156,17648,18395,19193,19200,21823,23395, 1989,  580,10022, 2628, 4663, 4821, 6233,12811,16855))
#   ORDER BY admissionid, start
# )
# SELECT
#     d.admissionid,
#     d.itemid,
#     d.item,
#     d.start, 
#     d.stop,
#     d.administered,
#     a.weightgroup,
#     a.heightgroup,
#     a.gender,
#     a.agegroup,
#     a.admittedat,
#     a.patientid,
#     a.dateofdeath,
#     f.admissionid,
#     f.ordercategory,
#     f.ordercategoryid,
#     f.start_time,
#     f.stop_time,
#     f.fluidin,
# FROM drugitems d
# LEFT JOIN admissions a ON
#      d.admissionid = a.admissionid
# LEFT JOIN fluid f ON
#      d.admissionid = f.admissionid
# WHERE d.itemid IN (
#         6818, -- Adrenaline (Epinefrine)
#         7178, -- Dobutamine (Dobutrex)
#         7179, -- Dopamine (Inotropin)
#         7229, -- Noradrenaline (Norepinefrine)
#         7291  -- Crystalloid (fluid)     
#     )
#     AND
#     (d.admissionid IN ( 6720,17191,17961, 4310,13925,12231, 1888,11884,  480,13611,  332,  125, 9284, 8865, 9651,12007,12008,10357, 1121, 3784, 7859,10358,15320,17652,19043,20137,20722, 2164,22537, 1927, 3958, 9406,11308,16789,20980, 9467,  751, 1317, 9092,23540,  460,  512, 9927,11542,15224, 3658, 7620, 9020,13441,16369,19066,21996, 8401,17696,14984,19130,22849, 1375, 1965, 1980, 2754, 3326, 3458, 4109, 4812, 4940, 5769, 6471, 7003, 8383, 8573,10350,14155,14571,17341,18339,19082,19134,19615,19689,19847,20219,21083,21247,21857,11319,22262, 2954, 3568, 6195, 8830,15681,15808,17077,20596, 1402, 4311, 5649, 6982, 2994, 4639,18759,22387,10771,19898, 4523,  223, 2049, 4429, 6590, 7772, 9986,10764,11483,12977,15204,15718,17264,17655,22427,22658, 2300,12733,23276,  252, 1914, 2215, 3073, 3105, 3333, 5225, 5832, 6959, 7351, 8295, 9041, 9810,10016,11069,11462,12141,13060,14512,17190,19663,   20,  208,  467,  733,  884,  892, 1084, 1108, 1677, 1692, 1900, 2193, 2307, 2381, 2397, 2494, 2692, 2933, 2953, 2982, 2999, 3365, 3558, 3630, 3802, 3831, 3874, 3891, 3908, 4002, 4080, 4193, 4347, 5164, 5525, 5908, 5979, 6034, 6131, 6158, 6242, 6251, 6673, 6763, 6862, 7254, 7441, 7841, 8043, 8652, 8753, 8814, 8936, 9252, 9254, 9345, 9471, 9546, 9849, 9871,10154,10553,10794,11079,11259,11289,11345,11905,11912,12113,12413,12565,12592,12660,12672,12781,12850,13182,13309,13346,13385,13421,13571,13681,13744,13846,13957,14175,14246,14314,14468,14549,14573,15489,15490,15610,15695,15738,15809,16048,16144,16161,16275,16341,16449,16707,16937,17770,18060,18070,18165,18188,18316,18413,18462,18509,18513,18652,18660,18696,18728,19069,19117,19241,19732,19886,19897,20176,20263,20493,20697,20854,20861,20942,21270,21450,21616,21858,21891,21973,21989,22038,22476,22571,22631,22655,22709,22719,22852,22874,23001,23212, 1680, 2755, 2879, 5928, 6310,10060,13348,13440,16422,17848, 2462, 2265, 2435, 2583, 3546, 7246,13315,15673,16562,  939, 5564, 6481, 7409,11858,14493,15845,15938,17506,19477,22780,22890,23050, 2411, 3074, 3152, 3627, 4168, 4731, 5871, 6033, 6411, 6589, 8298, 9605,11539,11828,12305,13802,15037,16513,16561,17562,18199,19478, 3368, 7284, 7285, 8705,13073,22202,22639,20591,  644, 2984, 3220, 3376, 3426, 3549, 3949, 4633, 5906, 6214,16013,16384,16388,16843,17111,19346,23373,  740, 7650, 8469, 8648, 9506,12126,16506,17360,19482,13261,  564, 1614, 5219, 6765, 7277, 9564, 9703,12956,15381,20545,20630,13532, 9895,10914, 1482, 2838, 3980, 5470, 6848, 7251,11920,21068,  446,  858,  953, 1296, 2010, 2845, 3190, 3319, 4484, 5498, 5656, 5697, 5897, 6551, 7059, 7981, 8427,10178,10600,12118,14474,14775,16211,18657,18673,21164,21781,22682,23150, 5840, 7667,15871,22699,  421, 5851,10896,11682,13814,14433,15304,20399,20558,21325,22142, 1309, 2589,16360,19000,19279, 2935,  159,  186,  243,  559,  801,  879,  888, 1679, 1756, 2264, 2365, 2479, 2648, 2750, 2791, 3010, 3327, 3450, 3735, 4038, 4355, 5387, 5651, 5711, 5721, 5937, 5964, 6151, 6736, 7047, 7376, 7726, 7931, 7985, 8088, 8279, 8367, 8521, 8562, 8717, 9194, 9818, 9842,10074,10644,10744,11002,11300,11433,11524,11675,11799,11866,11989,12042,12290,12318,12634,12806,12853,12866,12990,13063,13074,13239,13885,13963,14291,14382,14539,14887,15366,15383,16087,16378,16470,16632,16818,16875,17013,17479,17664,17694,17830,17833,17840,17931,18028,18131,18294,18308,18668,18863,18940,19590,19625,20307,20364,21028,21361,21657,22056,22356,22618,22674,22893,23031,23173,23328,23335,23336,23468, 9899,13072, 7662, 6582,13730,21710, 2600, 6970, 9717,12159,13761,17261,21895,23166, 8749,15937, 9636,15981,21934, 1374, 6253, 6898,10344,12158,17785,20584, 7256, 9146, 8874,10539,11239,15455,16996,22000,22340, 6860,21161, 1827,   25, 3905, 5251, 5829, 6122, 6640, 7504,14940,17629,20868,  576,  867, 4711, 8197, 8442, 8929,10023,14931,15442,15531,17651,17853,18622,19315,19970,20985,21059,21287, 1274, 6288,19262, 1845,15672,17168, 2807,18573, 1669, 3780, 5950,10056,10665,10735,13095,16061,16776,17025,17919,18678,22453,22966, 6377,18949,23388, 3031, 5347, 9922,13246,13929,14485,15805,16625,18734,21963,   70,  344,  502,  789,  988, 1003, 1392, 1758, 1804, 2096, 2550, 2604, 3405, 3758, 4094, 4202, 4837, 5000, 5113, 5170, 5263, 5477, 5848, 6421, 6865, 7286, 7680, 7937, 8740, 8765, 9368, 9597, 9654, 9914,10065,10121,10146,10208,10304,10409,10623,10909,10947,11051,11594,11612,11686,11916,11959,11993,12524,12697,12719,12740,12851,12991,13360,13480,13502,13738,13796,13984,14044,14192,14359,14557,14620,14941,15059,15064,15512,15771,15852,15991,16041,16280,16442,16454,16676,16689,16995,17150,17167,17612,17627,17638,17691,18646,18989,19435,19752,19856,20014,20344,20823,21228,21478,21808,22245,22313,22494,22568,23165,23225, 3362, 8261,14338,16683,17619,18393, 1353, 5089, 9255,12756,17903,19144,23470, 1135, 3136, 8691,11143,14939,17321,18504,19866,22823,23201, 4241, 7602, 8259,11293,12574,20388,20618,21571, 2482,15675,17079,21770,22608,  647, 4358,11807,13589,14344,15232,23462,22884, 5120, 5187,16758,22600,22767,23545,  213,12640,21243,23243, 1691, 2231, 6234,12981,18401,18898,21796,  560, 2151, 3579, 3785, 4886, 6102, 7689, 7863, 7988,13645,14646,17115,20115,  441, 3295, 4879, 7800, 8864, 9245,13517,13944,16365,16452,17459,22070,22603, 6645,   11,  165,  229,  245, 1602, 1613, 1727, 2024, 2237, 2285, 2346, 2581, 3160, 3300, 3390, 3840, 3997, 4015, 4163, 4500, 4535, 4668, 4673, 5008, 5019, 5335, 5442, 5557, 5789, 6055, 6238, 7235, 7460, 7823, 8041, 8096, 8594, 9382,10193,10401,10414,10843,10953,11177,11421,11893,11991,12211,12437,12922,13174,13299,13471,13545,13714,13766,14184,14339,14423,14630,14656,14805,14858,15086,15347,15662,15666,15827,16534,16656,16711,17049,17391,17517,17866,17922,18289,18517,18576,18725,18820,19436,19708,19950,20081,20211,20225,20411,21685,21752,21822,21860,21880,22144,22173,22435,22529,22673,22911,23341, 2829,14960,18368,19064,19424,22004,11690,  218,  685, 1262, 8255,14064,20083, 4799, 5107, 7010,11192, 9490,18451,  146,  859, 2350, 2835, 3129, 4921, 5203, 5866, 6100, 6983, 6990, 7786, 8802, 9410, 9889,11071,11341,14134,14335,16340,16627,16911,17936,22368,22916,   94, 1332, 2465, 5011, 5728, 6638, 6833, 6916, 7406, 7531, 7897, 8076, 8961, 9576, 9704,10036,11384,11445,11566,12492,13459,13719,13978,14615,16246,16326,16495,18522,19701,19727,19821,19966,20430,20976,21188,22261, 2277, 6152,  997, 1987, 4085,15886,19533,20323,  620, 2777, 2907, 7308,13269,20928, 1635, 2082, 2562, 6926, 6966, 7151, 8348, 8363, 9275,10009,12460,16230,21195,21865,22286, 5054, 8232,12960,14307,14545,23000, 1797, 6442, 8460, 9352,11265,11444, 8374,11408,11223, 4186,21286, 1070, 1449, 2580, 3715, 5579, 7630, 7643, 9670, 9774,11299,11579,13539,15317,15422,17852,18768,19354,19430,19444,20725,21466,21733,21803,21835,22518,23214,13835,16435, 3710, 4023, 6931, 8121, 8249,10166,10899,11017,15229,20993,  643, 7104, 8793, 9119,12193,12311,16244,18489,   66, 1115, 1180, 2980, 3989, 4770, 4802, 6053, 7669, 8212,11283,12345,13260,14363,14533,15877,15932,17078,17152,17662,17897,18109,18905,19521,21868,23164,10586,18812,19818,13387,  133, 8556, 9388,11090, 1278,21856, 2301, 2849,14342,20311,21824,12415, 1755, 2214, 9239,21442,  666, 2658, 5218, 7060,12473,13890,14670,17695,19267,20457,20851, 8453,15817,17058,20643, 4594, 6094,14437,15299,15961,19471,20313, 3396, 3682, 5531,21258,17498, 9535,14074, 2028, 3483, 3042,16276, 7340, 4350, 5409,12130,15501,20909,14499,15292, 1106, 3984, 6454, 9291,10918,11227,11736,12551,17716,18119,19748,22817, 5777,18787,22611,22837,20982, 1367,  348, 2189, 3918, 5775, 6635, 8990, 9181,11007,11570,13124,14072,20890,22927,18265,22998, 3928,15017,21642,19041, 8158,13755,16490,19277, 4505,15567, 4602, 5627,10512,14216,15363, 4042, 4622, 5221,11668,17403,  270, 1219, 8118,17469,17983,21683,22779,17823,17860,16915,17729,22132, 7905,14418,22348, 5619,12965, 4177, 6415, 7564,10214,23302, 2325, 3876,15914,14487,12933,18723,21277, 6210, 6642, 9494,12579,13704,20562, 2084, 3521, 8136, 1455, 9076,14632,14798,16090,22466, 4178,19202,22757, 5622, 6941,21647, 8321,11665,17760,17726, 5827, 9217,21155, 1595, 2893, 3016, 3661, 3706, 4715, 6112, 6181, 8634, 9693,11354,12932,13198,13458,14548,14851,16590,16778,17002,17672,19565,21626,22718, 6895, 6958,14142,  265,17100,11753,10992,12874, 2044, 2888, 3509, 3843, 5623, 6919, 7506, 7522, 9900,10446,10482,12723,13011,13991,15394,15571,15647,16165,17125,18068,18756,19155,12319,12542,23062, 9509, 4944,18206, 9385,14113,  203, 1310, 2054, 2657, 3301, 5740, 6924, 7349, 7632, 9852,11250,11465,12346,15680,17831,19804,22831,22832,13518,20272, 4791,12961, 2559, 2634, 5725,21754, 5540, 7293, 8853, 9397,10382,18873,20720,21753,  204, 7616, 7423,  495, 6945, 8618,10253,10331,12688,12734,13066,18633,20102,21049,21213,21406,22353, 3882, 1661, 3981,15557,18906,21533,12303, 4246, 4632, 5300, 8395,14675,15978,18959, 2422,15488,10532,12899, 5696, 7067, 7899,17951,15245,  225, 4216,12509,12510, 5299, 1032, 1361, 6709,17499, 8660,  188,  492,  992, 1437, 1522, 3296, 4034, 4398, 5736, 6235, 6351, 6793, 7432, 7839, 8035, 9007,10510,11729,11756,12974,13243,13642,15278,15849,16160,16206,17635,18150,18155,18291,18643,19582,19738,19814,19844,20005,20933,21810,22478,22543,23420, 5757,19790,22031,17071,  273,  274, 1112, 1456, 1590, 1715, 1724, 1732, 1809, 1958, 2368, 2402, 2669, 5127, 8054, 8560, 8927, 9438, 9804,10658,11119,11359,13087,13200,13489,13879,14518,16015,16836,16837,16883,17856,18407,19052,19074,19388,20557,21742,22771,22830,23361,23482, 1149, 2840, 8196, 9929,12296,15248,18807,21038, 8423, 7682,  210,  514, 3966, 4854, 7214, 7444, 8110,13617,16093,17389,18721,21134,22995, 1546, 1947, 6683, 7529, 7996,18140,19323,22454,22860,  720, 5560, 7056,15127,20066, 4205,10635,18298, 1315, 6355,14078,  676,  970, 1946, 4405, 4576, 5152, 5334, 5584, 5692, 6320, 6887, 7198, 7205, 8007, 8320, 8494, 8921, 8926, 9370, 9479,10150,11270,12578,13692,13933,14171,14631,14695,15423,16557,16720,18008,18292,18612,20138,20755,20768,21760,21906,22221,22267,22627,22717,23363,23488, 2052, 3856,13977,23190, 2952, 5003, 5572,10021,11559,  918, 9154,11982,20667,20816,17183,22548,23406,  977, 3756, 8382, 8440, 8848, 8857, 9244, 9801, 9976,10500,10930,12263,13251,13528,14882,14963,15910,18677,18699,18913,19542,20174,22160,13170, 2540, 4210, 8519,  863, 1434, 3018, 3598, 3814, 4925, 5153, 5573, 5706, 5716, 5802, 7949, 8021, 9726, 9742,10119,11778,11837,12023,14181,14994,16152,17084,17395,18749,19051,20216,20255,20516,20677,22166,22845,23472, 3159, 6853, 8751,11142,13783, 6331,17550,15275,17390,16138,  917, 2108, 3019, 3823, 4142, 7363,10845,11797,19091, 1457, 1681, 3686, 6604,11247,11409,11486,15578,16267,20268,22566,23282,23458, 7534,10622,22552,  645, 8145,13824,16770,17463, 1429, 9193,19329,22009,15942,  442,  919, 2789, 3699, 6016, 6499,10145,10317,10552,12645,12685,16301,17052,20341,20437,21727,23483, 9148, 9634,16304,17467,20356,20704,16820,17387,23140, 3742,15605,19356,  299, 8500, 9779,12897,13880,13903, 4189,13228,15370,12284, 7641,12654, 7177,20838,11307, 3533,19288,22145, 2070, 4923,16914, 6936,11059, 1674, 1169, 4083, 4524, 5624,10445,10524,11235,18415,22668,23193,   99, 1660,15466,15881,16850,17074,21396,21959,22641,23514,  774, 3857, 4353, 4572, 5753, 7482, 7599, 7888, 8592, 9010, 9912,11832,12371,12661,14115,15548,15555,15558,16549,17243,19494,19583,19805,20517,20602,21755,23304,17170, 3974, 7305,17673,16111,21209, 1348,15597, 3893, 8729, 8974,11145, 7663, 9903,11495,15566,16668,21515,22238, 1260, 1403, 3058, 5873, 6261, 6409, 6803, 7877,10235,10951,13013,13784,16329,17594,19138,20355,20413,20476,21119,22455,22672,22698,14165,15660,16173,20574,22501, 7468, 8174,10080,10215,15921,19938,11823,11970,16701, 4859, 4992, 9886,10342,20250, 1252, 3202, 3323,11072,18133,22207,10087,12003,16217,18950,18951,20811,13481,  318,  479,  976, 1411, 1847, 2115, 3059, 3111, 5283, 6271, 8142, 8963, 8977, 9463, 9473, 9748,10217,10625,11768,11824,12839,13020,13190,14389,14392,14397,14758,15694,16730,17134,17242,17428,17489,17883,18127,18714,18954,18955,19372,19495,20231,20293,21022,21401,21827,23171,11898,13953, 4835,15518,18249, 8216,15191,15632, 2457,12258,12313,13626,17314,19334, 3520,12662, 6436, 8250, 1502, 2046, 2569, 3986, 7025, 8945,10929,12844,16070,16552,21172,12255, 5397, 9328, 9530,11013,13788,22721, 2099, 4906,19395,  877, 2821, 5128,13221, 8677,15717,  272,  310, 2779, 3112, 3563, 4627, 4691, 4855, 4991, 5803, 5952, 7431, 7558, 9841, 9910, 9968,11095,11205,11551,12310,13040,13128,14770,15253,15529,16759,18290,18421,18882,19224,20106,22169,22687,23128,23372, 1102, 1237, 2284, 2946, 4019, 4053, 5034, 5069, 6714, 6826, 7554, 7976,14086,15454,15867,16468,16624,19940,20088,20573,20964,21130,21487,21873,22239,22887,12298,16533,  134,  547,  584,  800, 1013, 1018, 1105, 1123, 1547, 1688, 1754, 2483, 2769, 3068, 3140, 3336, 3656, 3855, 3910, 4065, 4696, 5427, 5975, 6050, 6276, 6668, 6676, 7019, 7021, 7134, 7319, 7337, 7370, 7576, 7914, 8005, 8247, 8372, 8517, 8540, 9107, 9206, 9247, 9349, 9440, 9512, 9662,10042,10242,10647,11025,11201,11260,11496,11666,11917,12577,12619,13090,13367,13734,13762,14055,14130,14145,14193,14421,14621,14643,14649,14876,14918,15396,15424,15617,15645,16201,16827,16967,17153,17241,17558,17762,17975,17995,18221,18790,19008,19090,19690,20011,20199,20251,20482,20812,20953,21168,21292,21392,21548,21611,21927,22104,22222,22257,22467,22694,22919,22923,23211,23476,23535, 1172, 8631,11248,12292,14700,17222,20107,20887, 3097, 2093, 6914,17182, 1513, 7391,14059,18010,20622,20139,20623,23186, 4476,12243,14018, 5401,12770,14496,15158,19245,20425,  108, 2050, 2466, 5695, 6820, 6883, 7402, 7447, 7774, 7955, 9182,10058,11769,12341,12529,12760,13955,14552,15528,15556,15616,17297,18661,18715,21510,22848,  957, 4624, 5100, 7474, 4766, 6675,12758,14469,   44,  896, 1027, 1464, 3008, 3498, 4561, 5284,10348,13562,14257,16168,19629, 5905, 6223, 8001, 8815,10368,12972,23480, 5542, 1336, 8417,11761,16017,17723, 2606, 3720, 8690,12192,12530,13273,16623,19563,  493,11748,21580,14896, 7040,14647,16448,20132, 9136,10505,14427,16941,21140,11006,20817,  160, 2147, 3006, 3783, 4889, 6071, 6196, 7961, 8122, 8686, 9806,10006,10976,11240,11634,13653,13902,15117,15706,16009,16248,17218,17720,18444,18869,22273,23155,23188, 1311, 2445, 2820, 2989, 4446, 4781, 5913, 6121, 8575, 8930,10112,10549,11082,11757,12244,13950,15111,15439,15517,17566,18009,18031,18301,23461,16129,   50,   91,  904, 1246, 1465, 2041, 2148, 2266, 3481, 3616, 4842, 4908, 5108, 5574, 5894, 6370, 6844, 7417, 7969, 8569, 8687, 8688,10323,10461,10829,11340,11691,11723,11947,12707,12761,13220,13521,13667,13690,13867,14019,14396,14479,14490,15081,15159,15446,15513,16427,17480,18058,18302,18446,18834,19164,19857,21465,21489,22134,22184,18597, 7623,10124, 5779, 1824, 2833,17849, 8277, 2473, 9409, 6641,18447, 3428, 4970,10943,  600, 2795,22862,14457, 1655, 1656, 1775, 4743,11150, 3056,20819,  677, 1879, 4346,16132,19610,20583,13934,15912,19081,  775, 7875, 4409, 7116, 3816,13410,22278, 2136,17082,12920, 9824,12449, 2205, 7379,17034,19253, 8187, 8643,12963,13700,22097,20585, 2579, 1749, 2188, 5458, 6201, 9322,11216,17466, 6546, 3076,11882,18582,  364,17616, 2955,12448,12927,18523,18391,19526, 1739,14090,22950,18870,14141, 8764,10326,19366, 6950, 1713, 9204,12336,19383, 4313,10343,13826,16517,  144, 7103,10341,10937, 6710,10597,21460, 1289, 2125, 3079,10088,10106,12281,12353,14862,16007,16612,17371,18216,19062,19717,21894,23382,11351,18347, 5382,14292, 6120,10000, 1618, 2853, 4410, 5206, 5439,12259,15611,15945,16733,18205,21154,21414,23075, 8834,11365,  403, 2553, 4805, 5440, 6005, 7446, 8741,12221,12470,13660,16672,21259,22596,22898, 5355,11685, 3104,10865,21874,18825, 1665,15713,17997,20053,20117, 3055,19217,19450, 1530, 1852, 2366,18225,18226, 7142,  387,18050,21199,  246, 5780, 6046, 7833, 8429, 8971,10051,10677,11010,11163,12246,20795,22805,22901,11349,13622, 1053, 7191,10381, 4727,11942,  901,  902, 5373, 6179,10395,14810, 5177, 5638, 7317,11490,12823,13270,14346,17407, 5313,  424, 2061, 2146, 2698, 3959, 5504, 7473, 9652,11864,15620,16584,16909,18358,19451,21064,21608,21678,23022, 2890,23330, 1419, 4466, 5590,16862,17081,12481,15709,19148,23465, 8360, 7157,12586,20916,20922,14005,15959,20703,21383,19403,14618, 1423, 8591,11166,21855, 6719, 2384, 8730,22058, 2556, 4422,16305,13601,  335,22506, 6038,10243,13597,19664,20872, 7963,18562,20684, 8087,20983, 3578, 1345, 2019, 4270, 5416, 6707, 6775, 7827, 8188, 8955,10260,11860,13874,16028,16122,16873,20792,21279, 9074,21303,15457,16771,19619,12878, 6508, 6509, 3044,16751,21953,10984, 5021, 9856,14462,17989,19621, 7361,11500,17231, 2122, 2963,12618,17324,18502, 3560,18963, 4098, 4986, 5046,11428,11610,12610, 6163,11789,13789,16594,13580,15418,19630, 5196, 5197, 8098,10905,15213,19902,20090,22715, 9019, 2541, 3577, 1120,11454,13395,14099,15207,16158,21920,22924,14030, 1864, 2766, 3584, 5904, 9018,11511,21997,22002,22981, 4195, 6566,12893,16429,18766,18809,20366,20575,21350,21503, 3897,10067,12219,19873,  625,  889, 1391, 1441, 1889, 2260, 3341, 3552, 3776, 4943, 5015, 6726, 6762, 8707, 8818, 8959, 9022, 9080, 9223,10877,10903,11616,13952,13988,15255,15561,15634,16742,19227,19316,19865,20649,21522,21828,21850, 1372, 5079, 4830, 6518,13407, 8720, 9319,17177,  510, 3515, 8859,10224,11414,12411,17216,14747, 4785,16964,21826, 2375, 7288,12241,15420,18890,18961,19449,21066, 2525, 4131, 8640, 8831,11382,16839,16936,17700,18432,19618,21242,14152,11237,14797, 1395, 1428, 4672,16098,16316,19280,19654,20690,17792,17807,  356,  414,  511, 2917, 5462, 7966, 8946,10310,11591,11698,12352,17405,18116,18821,19339,21351, 5261, 3659, 8394,10653,11284,11548,11592,13357,16405,18666,20926,21504,22409,23026, 5438,16219,  595,  865, 1856, 3291, 4477, 4679, 5605, 5708, 6291, 6708, 6773, 7080, 9398, 9650,11124,11567,14269,14703,16227,16979,17316,18962,19067,19835,20522,22349,16143, 8364, 9827, 2362,13205, 7781,12752,13093, 6275, 4078, 9675,12751,19710,21486,18281,14915,15768,16671,16947,19722, 5533, 5174,10740,16650,17669,  190, 8049,11242,21695, 2865,16481,   58, 5914,15968, 4974,20521,12300,14681,21597,22854, 6018,17837,19850, 7229, 3985,20352, 1161,14635,19943,  152,11987, 2428, 7119,16876,21251,22774,  715, 8482,12912, 2843, 3343,13408,13759,16072,13582, 3703, 2738, 3244, 6754,13708,14217,16406,19075,19251,20923,23079,23281, 8456, 4792, 9697,22405,   76, 6567, 8403,13804,16164,19162,21307,22870,18886, 5303, 9737,16095,16974, 1228,13776,20682,23141,12465, 7796, 8159,13565,19525,20331,14391,20357, 9620, 2114, 1555, 1762, 4518, 6066,10013,14223,15144,15460,16447,16747,17701,17746,18343,19705,20826,22481, 1224, 5792,12287,16354,22558, 1319, 8557,17332,21416,  388, 3089, 6892, 9286,10722,12873,16399, 9767,22337,   47,  880, 1412, 2524, 5910, 6735, 6903, 7892, 8190, 8410, 8531, 8839, 9381, 9542,10200,11232,12797,13305,13592,14224,15737,15987,16428,17156,17648,18395,19193,19200,21823,23395, 1989,  580,10022, 2628, 4663, 4821, 6233,12811,16855))

In [7]:
# action_space_query.to_csv('nieuw_action_space.csv')

# del(action_space_query)
# gc.collect()

In [8]:
# action_space_1 = pd.read_csv('action_space_0-12.csv')
# action_space_2 = pd.read_csv('action_space_12-24.csv')
# action_space_3 = pd.read_csv('action_space_24-36.csv')
# action_space_4 = pd.read_csv('action_space_36-48.csv')
# action_space_5 = pd.read_csv('action_space_48-60.csv')
# action_space_6 = pd.read_csv('action_space_60-72.csv')

In [9]:
# action_space_final = pd.concat([action_space_1, action_space_2, action_space_3, action_space_4, action_space_5, action_space_6])

In [10]:
# action_space_final.to_csv('nieuw_action_space.csv')

In [13]:
aki_query = pd.read_gbq(
    '''
WITH baseline AS (
    SELECT n.admissionid,
    MIN(n.value) AS baseline_creatinine
    FROM numericitems n
    LEFT JOIN admissions a ON
        n.admissionid = a.admissionid
    WHERE itemid IN (
        6836, --Kreatinine µmol/l (erroneously documented as µmol)
        9941, --Kreatinine (bloed) µmol/l
        14216 --KREAT enzym. (bloed) µmol/l
    ) AND
    (n.admissionid IN (
    '''+
    admission_ids+
    '''
    ))
    AND
    --search 1 year befor admission
    (n.measuredat - a.admittedat)/(60*60*1000) > -(365*24) AND (n.measuredat - a.admittedat) <= (72*60*60*1000)
    GROUP BY n.admissionid
),
max_creat AS (
    SELECT n.admissionid,
    MAX(n.value) AS max_creatinine
    FROM numericitems n
    LEFT JOIN admissions a ON
        n.admissionid = a.admissionid
    WHERE itemid IN (
        6836, --Kreatinine µmol/l (erroneously documented as µmol)
        9941, --Kreatinine (bloed) µmol/l
        14216 --KREAT enzym. (bloed) µmol/l
    )  AND
    (n.admissionid IN (
    '''+
    admission_ids+
    '''
    )) AND
    --within 72 hours of admission
    (n.measuredat - a.admittedat) > 0 AND (n.measuredat - a.admittedat) < (72*60*60*1000)
    GROUP BY n.admissionid
)
SELECT
    n.admissionid,
    n.itemid,
    n.item,
    n.value,
    n.registeredby,
    a.patientid,
    CASE
        WHEN n.registeredby NOT LIKE '%Systeem%' THEN TRUE
        ELSE FALSE
    END AS manual_entry,
    (n.measuredat - a.admittedat)/(1000*60) AS time,
    b.baseline_creatinine,
    m.max_creatinine,
    CASE
        -- AKI definition: 3 fold increase:
        WHEN baseline_creatinine > 0 AND m.max_creatinine/baseline_creatinine > 3 THEN TRUE
        -- AKI definition: increase to >= 354 umol/l AND at least 44 umol/l increase:
        WHEN max_creatinine >= 354 AND max_creatinine - baseline_creatinine >= 44 THEN TRUE
        ELSE FALSE
    END AS acute_renal_failure,
    CASE
        -- AKI Stage 1 Definition
        WHEN n.value >= 1.5*baseline_creatinine AND n.value < 2*baseline_creatinine THEN 1
        -- AKI Stage 2 Definition
        WHEN n.value >= 2*baseline_creatinine AND n.value < 3*baseline_creatinine THEN 2
        -- AKI Stage 3 Definition
        WHEN n.value >= 3*baseline_creatinine THEN 3
      END AS aki_stage
FROM numericitems n
LEFT JOIN admissions a ON
    n.admissionid = a.admissionid
LEFT JOIN baseline b ON -- get the baseline kreat (before admission)
    n.admissionid = b.admissionid
LEFT JOIN max_creat m ON --get the highest within a week of admission
    n.admissionid = m.admissionid
WHERE n.itemid IN (
    6836, --Kreatinine µmol/l (erroneously documented as µmol)
    9941, --Kreatinine (bloed) µmol/l
    14216 --KREAT enzym. (bloed) µmol/l
    ) AND
    (n.admissionid IN (
    '''+
    admission_ids+
    '''
    ))
-- measurements within 48 hours of ICU stay (use 30 minutes before admission to allow for time differences):
AND (n.measuredat - a.admittedat) <= 1000*60*60*72 AND (n.measuredat - a.admittedat) >= -(1000*60*30)
    '''
    , configuration=config_gbq)

In [14]:
aki_query

,admissionid,itemid,item,value,registeredby,patientid,manual_entry,time,baseline_creatinine,max_creatinine,acute_renal_failure,aki_stage
0,14493,9941,Kreatinine (bloed),84.0,Systeem,12503,False,3508.0,81.0,96.0,False,NaN
1,12551,9941,Kreatinine (bloed),87.0,Systeem,10831,False,1633.0,80.0,97.0,False,NaN
2,18302,9941,Kreatinine (bloed),106.0,Systeem,15798,False,3101.0,74.0,269.0,True,NaN
3,19471,9941,Kreatinine (bloed),34.0,Systeem,16800,False,1247.0,31.0,40.0,False,NaN
4,6763,9941,Kreatinine (bloed),131.0,Systeem,5841,False,45.0,85.0,131.0,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
14072,1927,9941,Kreatinine (bloed),95.0,Systeem,1676,False,642.0,52.0,104.0,False,1.0
14073,8188,9941,Kreatinine (bloed),97.0,Systeem,7059,False,899.0,94.0,103.0,False,NaN
14074,21515,9941,Kreatinine (bloed),191.0,Systeem,18570,False,1462.0,57.0,226.0,True,3.0
14075,19615,6836,Kreatinine,130.0,ICV_Medisch,16919,True,840.0,110.0,187.0,False,NaN


In [ ]:
aki_query.to_csv('nieuw_aki.csv')

# del(aki_query)
# gc.collect()

## 3. Extracted Data Statistics

Print some preliminary statistics about the extracted data

In [ ]:
print('Total Number of Sepsis Admissions: ', sepsis_counts['n_admissions'].sum())
print('Total Number of Unique Admissions in State Space: ', len(state_space_query['admissionid'].unique()))
print('Total Number of Unique Admissions in Action Space: ', len(action_space_query['admissionid'].unique()))
print('Total Number of Unique Admissions with Registered AKI: ', len(aki_query[aki_query['acute_renal_failure']== True]['admissionid'].unique()))